<a href="https://colab.research.google.com/github/jacobgreen4477/The-4th-ETRI-AI-Human-Understanding-Competition/blob/main/etri_new_baseline_v3_2_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> title : 제 4회 ETRI 휴먼이해 인공지능 논문경진대회 <br>
> author : hjy <br>

In our study, we used smartphones, smartwatches, sleep sensors, and self-recording apps to collect daily life logs and sleep health records of study participants in 2024.The data collection procedures and methods followed a similar approach to those used in previous studies. Here, we pu﻿blicly provide the following 12 data items, which comprise a total of 700 days' worth of lifelog data, strictly for non-commercial and academic research purposes only.
- mACStatus: Indicates whether the smartphone is currently being charged.
- mActivity: Value calculated by the Google Activity Recognition API.
- mAmbience: Ambient sound identification labels and their respective probabilities.
- mBle: Bluetooth devices around individual subject.
- mGps: Multiple GPS coordinates measured within a single minute using the smartphone.
- mLight: Ambient light measured by the smartphone.
- mScreenStatus: Indicates whether the smartphone screen is in use.
- mUsageStats: Indicates which apps were used on the smartphone and for how long.
- mWifi: Wifi devices around individual subject.
- wHr: Heart rate readings recorded by the smartwatch.
- wLight: Ambient light measured by the smartwatch.
- wPedo: Step data recorded by the smartwatch.

For the purpose of training a learning model to predict sleep health, fatigue, and stress, the following six metrics were derived from sleep sensor data and self-reported survey records. Each metric consists of values categorized into either two levels (0, 1) or three levels (0, 1, 2), depending on the specific metric. The detailed classification criteria for each metric's levels will be provided in a separate document.These
metrics assign a value of 0 for sleep records that do not meet the recommended guidelines.For instance, the first questionnaire metric (Q1) is assigned a value of 1 on days when an
individual’s self-reported sleep quality exceeds their average over the experimental period, and 0 when it
falls below that average. Similarly, the second and third metrics (Q2 and Q3) are assigned a value of 0
on days when the participant’s fatigue and stress levels, respectively, exceed their average, and a value of
1 when these levels are below average.

- Q1: Overall sleep quality as perceived by a subject immediately after waking up.
- Q2: Physical fatigue of a subject just before sleep.
- Q3: Stress level experienced by a subject just before sleep.
- S1: Adherence to sleep guidelines for total sleep time (TST).
- S2: Adherence to sleep guidelines for sleep efficiency (SE).
- S3: Adherence to sleep guidelines for sleep onset latency (SOL, or SL).

수면 건강, 피로, 스트레스 예측을 위한 학습 모델을 훈련시키기 위해, 수면 센서 데이터와 자기 보고식 설문 기록을 기반으로 다음의 6가지 지표를 도출했습니다.
각 지표는 해당 항목에 따라 두 수준(0, 1) 또는 세 수준(0, 1, 2)으로 구분된 값을 가집니다.
각 지표의 세부 분류 기준은 별도의 문서에서 제공될 예정입니다.

- Q1: 기상 직후 본인이 인지한 전반적인 수면의 질
 - 0: 개인 평균 이하
 - 1: 개인 평균 이상
- Q2: 취침 직전 본인이 느낀 신체적 피로 수준
 - 0: 높은 피로 수준
 - 1: 낮은 피로 수준
- Q3: 취침 직전 본인이 느낀 스트레스 수준
 - 0: 높은 스트레스 수준
 - 1: 낮은 스트레스 수준
- S1: 총 수면 시간(TST) 가이드라인을 준수했는지 3LEVELS
 - 0: 가이드라인 미준수
 - 1: 가이드라인 부분적 준수
 - 2: 가이드라인 완전 준수
- S2: 수면 효율(SE) 가이드라인을 준수했는지 여부
- (SE: 잠자리에 누워 있었던 전체 시간 대비, 실제로 잠든 시간의 비율)
 - 0: 가이드라인 미준수
 - 1: 가이드라인 준수
- S3: 수면 잠들기 지연 시간(SOL 또는 SL) 가이드라인을 준수했는지 여부
- (SOL: 잠자리에 누운 순간부터 실제로 잠드는 데까지 걸린 시간)
 - 0: 가이드라인 미준수
 - 1: 가이드라인 준수

# 📌 라이브러리

In [ ]:
! pip install haversine >/dev/null
! pip install optuna >/dev/null
! pip install category_encoders >/dev/null

# Core Libraries
import os
import sys
import re
import ast
import glob
import random
from io import StringIO
from collections import Counter
from datetime import datetime, timedelta, time

# Numerical Operations
import numpy as np
import pandas as pd

# Math & Geospatial
from math import radians, cos, sin, asin, sqrt
from scipy.stats import entropy
from haversine import haversine

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import (
    train_test_split, KFold, StratifiedKFold, cross_val_score
)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from lightgbm import LGBMClassifier, log_evaluation, early_stopping

# Deep Learning (PyTorch)
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F

# Progress Tracking
from tqdm import tqdm
from tqdm.auto import tqdm
from category_encoders import TargetEncoder

# Warnings
import warnings
warnings.filterwarnings('ignore')

# seed 고정
SD = 42
random.seed(SD)
np.random.seed(SD)
os.environ['PYTHONHASHSEED'] = str(SD)

# pandas 옵션
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%0.4f' % x)

In [ ]:
from sklearn.metrics import f1_score, log_loss

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
valid_ids1 = ['id012024-07-24', 'id012024-07-27', 'id012024-08-18', 'id012024-08-19', 'id012024-08-20', 'id012024-08-21', 'id012024-08-22', 'id012024-08-24', 'id012024-08-25', 'id012024-08-26', 'id012024-08-27', 'id012024-08-28', 'id012024-08-29', 'id012024-08-30', 'id022024-08-23', 'id022024-08-24', 'id022024-09-16', 'id022024-09-17', 'id022024-09-19', 'id022024-09-20', 'id022024-09-21', 'id022024-09-22', 'id022024-09-23', 'id022024-09-24', 'id022024-09-25', 'id022024-09-26', 'id022024-09-27', 'id022024-09-28', 'id032024-08-30', 'id032024-09-01', 'id032024-09-02', 'id032024-09-03', 'id032024-09-05', 'id032024-09-06', 'id032024-09-07', 'id042024-09-03', 'id042024-09-04', 'id042024-09-05', 'id042024-09-06', 'id042024-09-07', 'id042024-09-08', 'id042024-09-09', 'id042024-10-08', 'id042024-10-09', 'id042024-10-10', 'id042024-10-11', 'id042024-10-12', 'id042024-10-13', 'id042024-10-14', 'id052024-10-19', 'id052024-10-23', 'id052024-10-24', 'id052024-10-25', 'id052024-10-26', 'id052024-10-27', 'id052024-10-28', 'id062024-07-25', 'id062024-07-26', 'id062024-07-27', 'id062024-07-28', 'id062024-07-29', 'id062024-07-30', 'id062024-07-31', 'id072024-07-07', 'id072024-07-08', 'id072024-07-09', 'id072024-07-10', 'id072024-07-11', 'id072024-07-12', 'id072024-07-13', 'id072024-07-30', 'id072024-08-01', 'id072024-08-02', 'id072024-08-03', 'id072024-08-04', 'id072024-08-05', 'id072024-08-06', 'id082024-08-28', 'id082024-08-29', 'id082024-08-30', 'id082024-08-31', 'id082024-09-01', 'id082024-09-02', 'id082024-09-04', 'id092024-08-02', 'id092024-08-22', 'id092024-08-23', 'id092024-08-24', 'id092024-08-25', 'id092024-08-27', 'id092024-08-28', 'id092024-08-29', 'id092024-08-30', 'id092024-08-31', 'id092024-09-01', 'id092024-09-02', 'id092024-09-03', 'id092024-09-04', 'id102024-08-28', 'id102024-08-30', 'id102024-08-31', 'id102024-09-01', 'id102024-09-02', 'id102024-09-03', 'id102024-09-06']
valid_ids2 = ['id012024-07-24', 'id012024-07-27', 'id012024-08-19', 'id012024-08-20', 'id012024-08-21', 'id012024-08-22', 'id012024-08-24', 'id012024-08-25', 'id012024-08-26', 'id012024-08-27', 'id012024-08-28', 'id012024-08-29', 'id012024-08-30', 'id012024-09-01', 'id022024-08-23', 'id022024-08-24', 'id022024-09-13', 'id022024-09-14', 'id022024-09-16', 'id022024-09-17', 'id022024-09-19', 'id022024-09-20', 'id022024-09-21', 'id022024-09-22', 'id022024-09-23', 'id022024-09-24', 'id022024-09-25', 'id022024-09-26', 'id032024-09-02', 'id032024-09-03', 'id032024-09-05', 'id032024-09-06', 'id032024-09-07', 'id032024-09-08', 'id042024-09-07', 'id042024-09-08', 'id042024-09-09', 'id042024-09-11', 'id042024-09-17', 'id042024-09-18', 'id042024-09-28', 'id042024-09-29', 'id042024-10-21', 'id042024-10-23', 'id042024-10-27', 'id052024-08-29', 'id052024-08-30', 'id052024-08-31', 'id052024-09-01', 'id052024-10-10', 'id052024-11-05', 'id052024-11-06', 'id052024-11-10', 'id052024-11-11', 'id052024-11-12', 'id052024-11-15', 'id062024-08-03', 'id062024-08-04', 'id062024-08-05', 'id062024-08-06', 'id062024-08-11', 'id062024-08-16', 'id062024-08-19', 'id072024-07-02', 'id072024-07-03', 'id072024-07-04', 'id072024-07-06', 'id072024-07-07', 'id072024-07-08', 'id072024-07-09', 'id072024-08-02', 'id072024-08-03', 'id072024-08-04', 'id072024-08-05', 'id072024-08-06', 'id072024-08-07', 'id072024-08-08', 'id082024-09-01', 'id082024-09-02', 'id082024-09-04', 'id082024-09-06', 'id082024-09-12', 'id082024-09-16', 'id082024-09-17', 'id092024-07-27', 'id092024-07-28', 'id092024-07-30', 'id092024-07-31', 'id092024-08-02', 'id092024-08-04', 'id092024-08-05', 'id092024-08-22', 'id092024-08-23', 'id092024-08-24', 'id092024-08-25', 'id092024-08-27', 'id092024-08-28', 'id092024-08-29', 'id102024-08-30', 'id102024-08-31', 'id102024-09-01', 'id102024-09-02', 'id102024-09-03', 'id102024-09-06', 'id102024-09-08']
valid_ids3 = ['id012024-07-20', 'id012024-07-23', 'id012024-08-19', 'id012024-08-20', 'id012024-08-21', 'id012024-08-22', 'id012024-08-24', 'id012024-08-25', 'id012024-08-26', 'id012024-08-27', 'id012024-08-28', 'id012024-08-29', 'id012024-08-30', 'id012024-09-01', 'id022024-08-21', 'id022024-08-22', 'id022024-09-11', 'id022024-09-12', 'id022024-09-13', 'id022024-09-14', 'id022024-09-16', 'id022024-09-17', 'id022024-09-19', 'id022024-09-20', 'id022024-09-21', 'id022024-09-22', 'id022024-09-23', 'id022024-09-24', 'id032024-09-05', 'id032024-09-06', 'id032024-09-07', 'id032024-09-08', 'id032024-09-10', 'id032024-09-12', 'id032024-09-13', 'id042024-08-27', 'id042024-08-28', 'id042024-08-29', 'id042024-08-30', 'id042024-08-31', 'id042024-09-01', 'id042024-09-02', 'id042024-10-01', 'id042024-10-02', 'id042024-10-03', 'id042024-10-04', 'id042024-10-05', 'id042024-10-06', 'id042024-10-07', 'id052024-10-28', 'id052024-10-29', 'id052024-10-30', 'id052024-10-31', 'id052024-11-03', 'id052024-11-05', 'id052024-11-06', 'id062024-07-31', 'id062024-08-01', 'id062024-08-02', 'id062024-08-03', 'id062024-08-04', 'id062024-08-05', 'id062024-08-06', 'id072024-06-29', 'id072024-06-30', 'id072024-07-01', 'id072024-07-02', 'id072024-07-03', 'id072024-07-04', 'id072024-07-06', 'id072024-08-07', 'id072024-08-08', 'id072024-08-09', 'id072024-08-10', 'id072024-08-11', 'id072024-08-12', 'id072024-08-13', 'id082024-08-19', 'id082024-08-20', 'id082024-08-22', 'id082024-08-23', 'id082024-08-24', 'id082024-08-25', 'id082024-08-26', 'id092024-08-04', 'id092024-08-22', 'id092024-08-23', 'id092024-08-24', 'id092024-08-25', 'id092024-08-27', 'id092024-08-28', 'id092024-08-29', 'id092024-08-30', 'id092024-08-31', 'id092024-09-01', 'id092024-09-02', 'id092024-09-03', 'id092024-09-04', 'id102024-09-02', 'id102024-09-03', 'id102024-09-06', 'id102024-09-08', 'id102024-09-09', 'id102024-09-12', 'id102024-09-15']
valid_ids4 = ['id012024-07-24', 'id012024-07-27', 'id012024-08-18', 'id012024-08-19', 'id012024-08-20', 'id012024-08-21', 'id012024-08-22', 'id012024-08-24', 'id012024-08-25', 'id012024-08-26', 'id012024-08-27', 'id012024-08-28', 'id012024-08-29', 'id012024-08-30', 'id022024-08-23', 'id022024-08-24', 'id022024-09-12', 'id022024-09-13', 'id022024-09-14', 'id022024-09-16', 'id022024-09-17', 'id022024-09-22', 'id022024-09-23', 'id022024-09-24', 'id022024-09-25', 'id022024-09-26', 'id022024-09-27', 'id022024-09-28', 'id032024-08-30', 'id032024-09-01', 'id032024-09-02', 'id032024-09-07', 'id032024-09-08', 'id032024-09-10', 'id042024-09-03', 'id042024-09-04', 'id042024-09-05', 'id042024-09-11', 'id042024-09-17', 'id042024-09-18', 'id042024-09-28', 'id042024-09-29', 'id042024-10-21', 'id042024-10-23', 'id042024-10-27', 'id052024-08-29', 'id052024-08-30', 'id052024-08-31', 'id052024-09-01', 'id052024-10-10', 'id052024-11-03', 'id052024-11-05', 'id052024-11-10', 'id052024-11-11', 'id052024-11-12', 'id052024-11-15', 'id062024-07-27', 'id062024-07-28', 'id062024-07-29', 'id062024-07-30', 'id062024-08-11', 'id062024-08-16', 'id062024-08-19', 'id072024-07-03', 'id072024-07-04', 'id072024-07-06', 'id072024-07-10', 'id072024-07-11', 'id072024-07-12', 'id072024-07-13', 'id072024-07-29', 'id072024-07-30', 'id072024-08-01', 'id072024-08-02', 'id072024-08-03', 'id072024-08-04', 'id072024-08-05', 'id082024-09-01', 'id082024-09-02', 'id082024-09-04', 'id082024-09-06', 'id082024-09-12', 'id082024-09-16', 'id082024-09-17', 'id092024-07-02', 'id092024-07-04', 'id092024-07-05', 'id092024-07-06', 'id092024-08-02', 'id092024-08-04', 'id092024-08-05', 'id092024-08-22', 'id092024-08-23', 'id092024-08-24', 'id092024-08-25', 'id092024-08-27', 'id092024-08-28', 'id092024-08-29', 'id102024-07-27', 'id102024-07-28', 'id102024-07-29', 'id102024-07-30', 'id102024-08-01', 'id102024-08-02', 'id102024-08-03']

# 📌 데이터 전처리

### 📦 데이터 읽기

In [ ]:
path = '/content/drive/MyDrive/data/ch2025_data_items/'

# 1
mACStatus = pd.read_parquet(path+'ch2025_mACStatus.parquet')
mActivity = pd.read_parquet(path+'ch2025_mActivity.parquet')
mAmbience = pd.read_parquet(path+'ch2025_mAmbience.parquet')
mBle = pd.read_parquet(path+'ch2025_mBle.parquet')
mGps = pd.read_parquet(path+'ch2025_mGps.parquet')
mLight = pd.read_parquet(path+'ch2025_mLight.parquet')
mScreenStatus = pd.read_parquet(path+'ch2025_mScreenStatus.parquet')
mUsageStats = pd.read_parquet(path+'ch2025_mUsageStats.parquet')
mWifi = pd.read_parquet(path+'ch2025_mWifi.parquet')
wHr = pd.read_parquet(path+'ch2025_wHr.parquet')
wLight = pd.read_parquet(path+'ch2025_wLight.parquet')
wPedo = pd.read_parquet(path+'ch2025_wPedo.parquet')

# 2
train = pd.read_csv('/content/drive/MyDrive/data/ch2025_metrics_train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/ch2025_submission_sample.csv')

### ✔️ mACStatus 핸드폰 충전상태
- Indicates whether the smartphone is currently being charged.
- m_charging : 0/1 상태
- 핸드폰이 오랫 동안 충전했다는 의미?
 - 한 자리에 장시간 머물러 있었다.
 - 핸드폰을 장시간 사용하지 않았다.  

In [ ]:
mACStatus['lifelog_date'] = mACStatus['timestamp'].astype(str).str[:10]
mACStatus.head(1)

,subject_id,timestamp,m_charging,lifelog_date
0,id01,2024-06-26 12:03:00,0,2024-06-26


In [ ]:
def process_mACStatus(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])

    results = []

    for (subj, lifelog_date), group in df.groupby(['subject_id', 'lifelog_date']):
        status = group['m_charging'].values  # 0/1 상태
        times = group['timestamp'].values

        ratio_charging = status.mean()
        sum_charging = status.sum()

        # 상태 전이 횟수
        transitions = (status[1:] != status[:-1]).sum()

        # 연속된 1 상태 길이들
        lengths = []
        current_len = 0
        for val in status:
            if val == 1:
                current_len += 1
            elif current_len > 0:
                lengths.append(current_len)
                current_len = 0
        if current_len > 0:
            lengths.append(current_len)

        avg_charging_duration = np.mean(lengths) if lengths else 0
        max_charging_duration = np.max(lengths) if lengths else 0

        results.append({
            'subject_id': subj,
            'lifelog_date': lifelog_date,
            'charging_ratio': ratio_charging,
            'charging_sum': sum_charging,
            'charging_transitions': transitions,
            'avg_charging_duration': avg_charging_duration,
            'max_charging_duration': max_charging_duration,
        })

    return pd.DataFrame(results)

mACStatus2 = process_mACStatus(mACStatus)

# check
print(f'# mACStatus2 shape: {mACStatus2.shape}')
mACStatus2.head(1)

# mACStatus2 shape: (700, 7)


,subject_id,lifelog_date,charging_ratio,charging_sum,charging_transitions,avg_charging_duration,max_charging_duration
0,id01,2024-06-26,0.2159,147,22,13.3636,41


### ✔️ mActivity 추정행동
- Value calculated by the Google Activity Recognition API.
 - 0 : IN_VEHICLE
 - 1 : ON_BICYCLE
 - 2 : ON_FOOT
 - 3 : STILL (not moving)
 - 4 : UNKNOWN
 - 5 : TILTING (This often occurs when a device is picked up from a desk or a user who is sitting stands up.)
 - 7 : WALKING
 - 8 : RUNNING
- 근무시간   : 오전 7시부터 오후 6시까지
- 근무외시간 : 오후6시부터 12시까지

In [ ]:
mActivity['lifelog_date'] = mActivity['timestamp'].astype(str).str[:10]
mActivity.head()

,subject_id,timestamp,m_activity,lifelog_date
0,id01,2024-06-26 12:03:00,4,2024-06-26
1,id01,2024-06-26 12:04:00,0,2024-06-26
2,id01,2024-06-26 12:05:00,0,2024-06-26
3,id01,2024-06-26 12:06:00,0,2024-06-26
4,id01,2024-06-26 12:07:00,0,2024-06-26


In [ ]:
def process_mActivity(df):
    """
    # 포함
    - 0 : IN_VEHICLE
    - 1 : ON_BICYCLE
    - 2 : ON_FOOT
    - 5 : TILTING (This often occurs when a device is picked up from a desk or a user who is sitting stands up.)
    - 7 : WALKING
    - 8 : RUNNING

    # 제외
    - 3 : STILL (not moving)
    - 4 : UNKNOWN
    """
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['hour'] = df['timestamp'].dt.hour

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'lifelog_date']):
        row = {'subject_id': subj, 'lifelog_date': date}

        # 전체 시간에서 1,2,7,8
        a1 = group[group['m_activity'].isin([1,2,7,8])]
        row['all_WALKING_n_ETC_minutes'] = len(a1)

        # 전체 시간에서 0 (IN_VEHICLE)
        a2 = group[group['m_activity'].isin([0])]
        row['all_VEHICLE_minutes'] = len(a2)

        # 전체 시간에서 유효한 활동
        all_valid = group[group['m_activity'].isin([0, 1, 2, 5, 7, 8])]
        row['all_ACTIVITY_minutes'] = len(all_valid)

        # sleeptime 0~5시 에서 유효한 활동
        dawn_valid = all_valid[(all_valid['hour'] >= 0) & (all_valid['hour'] <5)]
        row['dawn_ACTIVITY_minutes'] = len(dawn_valid)

        results.append(row)

    return pd.DataFrame(results)

In [ ]:
mActivity2 = process_mActivity(mActivity)

# check
print(f'# mActivity2 shape: {mActivity2.shape}')
mActivity2.head(1)

# mActivity2 shape: (700, 6)


,subject_id,lifelog_date,all_WALKING_n_ETC_minutes,all_VEHICLE_minutes,all_ACTIVITY_minutes,dawn_ACTIVITY_minutes
0,id01,2024-06-26,32,89,121,0


### ✔️ mAmbience 추정주변소리
- Ambient sound identification labels and their respective probabilities.
- 무슨 소리가 난게 중요할까?
- 새벽에 무슨 소리던지 소리가 난게 중요한 걸까?
- 여러 가지 소리 중에 노이즈도 포함되어 있을까?

In [ ]:
def extract_labels_and_probs(row):
    items = row['m_ambience']
    labels = [item[0] for item in items]
    probs = [item[1] for item in items]
    return pd.Series({'labels': labels, 'prob': probs})

mAmbience[['labels', 'prob']]  = mAmbience.apply(extract_labels_and_probs, axis=1)
mAmbience['lifelog_date'] = mAmbience['timestamp'].astype(str).str[:10]
mAmbience = mAmbience.drop(columns=['m_ambience'])
mAmbience.head(1)

,subject_id,timestamp,labels,prob,lifelog_date
0,id01,2024-06-26 13:00:10,"[Music, Vehicle, Motor vehicle (road), Outside, urban or manmade, Outside, rural or natural, Car, Speech, Inside, large room or hall, Truck, Sound effect]","[0.30902618, 0.081680894, 0.04035286, 0.037144363, 0.032663062, 0.03199804, 0.029806137, 0.01684492, 0.016206821, 0.01591479]",2024-06-26


In [ ]:
def process_mAmbience(df):

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['hour'] = df['timestamp'].dt.hour

    # 시간대 분류
    df['time_period'] = df['hour'].apply(lambda h: 'sleeptime' if 0 <= h < 5 else 'activehour')

    # explode labels
    exploded = df.explode('labels')

    # unique label count
    unique_labels = (
        exploded.groupby(['subject_id', 'lifelog_date', 'time_period'])['labels']
        .nunique()
        .reset_index(name='unique_label_count')
    )

    # snor 포함 라벨 count
    snor_labels = (
        exploded[exploded['labels'].astype(str).str.contains('snor', case=False, na=False)]
        .groupby(['subject_id', 'lifelog_date', 'time_period'])['labels']
        .count()
        .reset_index(name='snor_count')
    )

    # 병합
    result = pd.merge(unique_labels, snor_labels, on=['subject_id', 'lifelog_date', 'time_period'], how='outer').fillna(0)
    result['snor_count'] = result['snor_count'].astype(int)
    result = result.pivot(index=['subject_id', 'lifelog_date'], columns='time_period')
    result.columns = [f"{tp}_{metric}" for metric, tp in result.columns]
    result = result.reset_index()

    return result

In [ ]:
mAmbience2 = process_mAmbience(mAmbience)

# check
print(f'# mAmbience2 shape: {mAmbience2.shape}')
mAmbience2.head(1)

# mAmbience2 shape: (700, 6)


,subject_id,lifelog_date,activehour_unique_label_count,sleeptime_unique_label_count,activehour_snor_count,sleeptime_snor_count
0,id01,2024-06-26,265.0000,NaN,4.0000,NaN


### ✔️ mBle 블루투스
- Bluetooth devices around individual subject.
 - 7936 : Wearable, Headset, AV Device
 - 1796 : Peripheral (입력장치) 계열
 - 0 : 정보 없음 또는 알 수 없음(Unknown)
 - 1084 : Audio/Video (스피커, 헤드셋, 이어폰, TV 등)
 - 524 : Phone (휴대폰, 스마트폰)
 - 1060 : Headphones
 - 284 : commputer (PC, 노트북, PDA)

In [ ]:
def extract_mble_info(row):
    m_data = row['m_ble']
    address = [item['address'] for item in m_data]
    device_class = [item['device_class'] for item in m_data]
    rssi = [item['rssi'] for item in m_data]
    return pd.Series({'address': address, 'device_class': device_class, 'rssi': rssi})

mBle[['address','device_class','rssi']] = mBle.apply(extract_mble_info, axis=1)
mBle['lifelog_date'] = mBle['timestamp'].astype(str).str[:10]
mBle.head(1)

,subject_id,timestamp,m_ble,address,device_class,rssi,lifelog_date
0,id01,2024-06-26 12:13:00,"[{'address': '00:15:7C:11:80:8D', 'device_class': '0', 'rssi': -82}, {'address': '01:B1:D2:20:9E:3A', 'device_class': '0', 'rssi': -61}, {'address': '04:33:1F:D9:C1:50', 'device_class': '0', 'rssi': -86}, {'address': '06:5C:2D:BC:39:BE', 'device_class': '0', 'rssi': -75}, {'address': '09:42:21:0D:AD:DF', 'device_class': '0', 'rssi': -70}, {'address': '0B:66:0D:D5:9C:4A', 'device_class': '0', 'rssi': -89}, {'address': '10:B5:88:E7:85:69', 'device_class': '0', 'rssi': -89}, {'address': '13:F0:CA:3B:DB:EF', 'device_class': '0', 'rssi': -77}, {'address': '1A:23:C0:8F:43:4D', 'device_class': '0', 'rssi': -66}, {'address': '24:11:53:BB:62:89', 'device_class': '1796', 'rssi': -37}, {'address': '24:2D:F0:EE:1E:D0', 'device_class': '0', 'rssi': -85}, {'address': '26:0C:48:28:15:77', 'device_class': '0', 'rssi': -63}, {'address': '27:C1:C0:8B:82:C9', 'device_class': '0', 'rssi': -88}, {'address': '28:9C:11:73:39:05', 'device_class': '0', 'rssi': -30}, {'address': '34:40:DE:35:F8:65', 'device_class': '0', 'rssi': -93}, {'address': '35:0A:59:BF:75:F5', 'device_class': '0', 'rssi': -72}, {'address': '41:A6:C4:20:E3:2C', 'device_class': '7936', 'rssi': -83}, {'address': '42:6B:51:95:1B:D4', 'device_class': '0', 'rssi': -77}, {'address': '44:B2:0B:78:04:0F', 'device_class': '0', 'rssi': -69}, {'address': '45:37:48:E2:7F:CC', 'device_class': '0', 'rssi': -87}, {'address': '4E:1B:C2:DF:C5:87', 'device_class': '0', 'rssi': -76}, {'address': '4E:9F:1B:A9:56:5D', 'device_class': '0', 'rssi': -66}, {'address': '50:63:B0:82:07:00', 'device_class': '0', 'rssi': -86}, {'address': '53:13:6C:4F:04:D2', 'device_class': '0', 'rssi': -69}, {'address': '54:15:89:95:27:44', 'device_class': '7936', 'rssi': -71}, {'address': '56:0E:2E:B0:D4:11', 'device_class': '0', 'rssi': -61}, {'address': '5A:7A:2E:42:03:B1', 'device_class': '0', 'rssi': -82}, {'address': '5A:9D:3E:AB:38:C6', 'device_class': '0', 'rssi': -83}, {'address': '5E:A6:8E:B8:74:74', 'device_class': '0', 'rssi': -84}, {'address': '5F:BC:08:0F:C1:6A', 'device_class': '0', 'rssi': -87}, {'address': '62:E1:9D:41:F4:AE', 'device_class': '0', 'rssi': -73}, {'address': '67:23:FE:88:69:A8', 'device_class': '0', 'rssi': -88}, {'address': '68:EC:C5:0C:D1:C1', 'device_class': '0', 'rssi': -78}, {'address': '6B:28:DA:C0:1B:29', 'device_class': '0', 'rssi': -75}, {'address': '6F:0B:91:00:33:19', 'device_class': '0', 'rssi': -80}, {'address': '70:7A:4B:82:44:90', 'device_class': '0', 'rssi': -88}, {'address': '7B:62:D4:5B:59:D3', 'device_class': '0', 'rssi': -74}, {'address': '7B:BE:A4:9D:FD:11', 'device_class': '0', 'rssi': -72}, {'address': '7F:FD:C4:00:77:7D', 'device_class': '0', 'rssi': -52}, {'address': 'C4:F0:92:C8:F1:8D', 'device_class': '7936', 'rssi': -87}, {'address': 'C7:3F:2C:7B:86:66', 'device_class': '7936', 'rssi': -89}]","[00:15:7C:11:80:8D, 01:B1:D2:20:9E:3A, 04:33:1F:D9:C1:50, 06:5C:2D:BC:39:BE, 09:42:21:0D:AD:DF, 0B:66:0D:D5:9C:4A, 10:B5:88:E7:85:69, 13:F0:CA:3B:DB:EF, 1A:23:C0:8F:43:4D, 24:11:53:BB:62:89, 24:2D:F0:EE:1E:D0, 26:0C:48:28:15:77, 27:C1:C0:8B:82:C9, 28:9C:11:73:39:05, 34:40:DE:35:F8:65, 35:0A:59:BF:75:F5, 41:A6:C4:20:E3:2C, 42:6B:51:95:1B:D4, 44:B2:0B:78:04:0F, 45:37:48:E2:7F:CC, 4E:1B:C2:DF:C5:87, 4E:9F:1B:A9:56:5D, 50:63:B0:82:07:00, 53:13:6C:4F:04:D2, 54:15:89:95:27:44, 56:0E:2E:B0:D4:11, 5A:7A:2E:42:03:B1, 5A:9D:3E:AB:38:C6, 5E:A6:8E:B8:74:74, 5F:BC:08:0F:C1:6A, 62:E1:9D:41:F4:AE, 67:23:FE:88:69:A8, 68:EC:C5:0C:D1:C1, 6B:28:DA:C0:1B:29, 6F:0B:91:00:33:19, 70:7A:4B:82:44:90, 7B:62:D4:5B:59:D3, 7B:BE:A4:9D:FD:11, 7F:FD:C4:00:77:7D, C4:F0:92:C8:F1:8D, C7:3F:2C:7B:86:66]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1796, 0, 0, 0, 0, 0, 0, 7936, 0, 0, 0, 0, 0, 0, 0, 7936, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7936, 7936]","[-82, -61, -86, -75, -70, -89, -89, -77, -66, -37, -85, -63, -88, -30, -93, -72, -83, -77, -69, -87, -76, -66, -86, -69, -71, -61, -82, -83, -84, -87, -73, -88, -78, -75, -80, 

In [ ]:
def process_mBle(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour

    # 시간대 분류
    def map_time_period(row):
        if 0 <= row['hour'] < 7:
            return 'sleeptime'
        elif 7 <= row['hour'] < 18:
            return 'worktime'
        else:
            return 'afterwork'

    df['time_period'] = df.apply(map_time_period, axis=1)

    features = []

    for idx, row in df.iterrows():
        entry = ast.literal_eval(row['m_ble']) if isinstance(row['m_ble'], str) else row['m_ble']

        rssi_list = []
        class_0_cnt = 0
        class_other_cnt = 0

        for device in entry:
            try:
                rssi = int(device['rssi'])
                rssi_list.append(rssi)

                device_class = str(device['device_class'])
                if device_class == '0':
                    class_0_cnt += 1
                else:
                    class_other_cnt += 1
            except:
                continue  # malformed record

        feature = {
            'subject_id': row['subject_id'],
            'lifelog_date': row['lifelog_date'],
            'time_period': row['time_period'],
            'ble_class_unknwn_cnt': class_0_cnt,
            'ble_class_others_cnt': class_other_cnt,
            'ble_count': len(rssi_list),
            'ble_rssi_mean': np.mean(rssi_list) if rssi_list else np.nan,
            'ble_rssi_min': np.min(rssi_list) if rssi_list else np.nan,
            'ble_rssi_max': np.max(rssi_list) if rssi_list else np.nan,
        }
        features.append(feature)

    return pd.DataFrame(features)

def summarize_mBle_daily(df):

    # row 단위 BLE feature 추출
    df = process_mBle(df)

    # 하루 + 시간대별로 groupby
    grouped = df.groupby(['subject_id', 'lifelog_date', 'time_period']).agg({
        'ble_class_unknwn_cnt': 'sum',
        'ble_class_others_cnt': 'sum',
        'ble_rssi_mean': 'mean',
        'ble_rssi_min': 'min',
        'ble_rssi_max': 'max',
    }).reset_index()

    # 총합 구해서 비율 계산
    total_cnt = grouped['ble_class_unknwn_cnt'] + grouped['ble_class_others_cnt']
    grouped['ble_class_unknwn_ratio'] = grouped['ble_class_unknwn_cnt'] / total_cnt.replace(0, np.nan)
    grouped['ble_class_others_ratio'] = grouped['ble_class_others_cnt'] / total_cnt.replace(0, np.nan)

    # 필요 없는 cnt 컬럼 제거
    grouped.drop(columns=[
        'ble_class_unknwn_cnt',
        'ble_class_others_cnt'
    ], inplace=True)

    # pivot해서 time_period별로 펼치기
    final = grouped.pivot(index=['subject_id', 'lifelog_date'], columns='time_period')
    final.columns = ['_'.join(col).strip() for col in final.columns.values]
    final = final.reset_index()

    return final

In [ ]:
mBle2 = summarize_mBle_daily(mBle)

# check
print(f'\n # mBle2 shape: {mBle2.shape}')
mBle2.head(1)


 # mBle2 shape: (651, 17)


,subject_id,lifelog_date,ble_rssi_mean_afterwork,ble_rssi_mean_sleeptime,ble_rssi_mean_worktime,ble_rssi_min_afterwork,ble_rssi_min_sleeptime,ble_rssi_min_worktime,ble_rssi_max_afterwork,ble_rssi_max_sleeptime,ble_rssi_max_worktime,ble_class_unknwn_ratio_afterwork,ble_class_unknwn_ratio_sleeptime,ble_class_unknwn_ratio_worktime,ble_class_others_ratio_afterwork,ble_class_others_ratio_sleeptime,ble_class_others_ratio_worktime
0,id01,2024-06-26,-76.2155,NaN,-75.0522,-92.0000,NaN,-94.0000,-43.0000,NaN,-27.0000,0.9237,NaN,0.9421,0.0763,NaN,0.0579


### ✔️ mGps, GPS 기반 핸드폰 위치
- Multiple GPS coordinates measured within a single minute using the smartphone.
- speed가 1보다 큰경우 정지 상태가 아니고 움직이고 있다고 판단
 - 0.5-2 : 걸어서 이동하는 경우  
 - 2-5 : 조깅
 - 5 이상 : 차를 타고 이동하는 경우

- speed가 0.5-2사이를 하루에 몇분동안 지속했는지?
- speed가 2-5사이를 하루에 몇분동안 지속했는지? (유산소 운동 시간)
- speed가 5이상을 하루에 몇분동안 지속했는지?  

In [ ]:
def extract_gps_info(row):
    m_data = row['m_gps']
    altitude = [item['altitude'] for item in m_data]
    latitude = [item['latitude'] for item in m_data]
    longitude = [item['longitude'] for item in m_data]
    speed = [item['speed'] for item in m_data]
    return pd.Series({'altitude': altitude, 'latitude': latitude, 'longitude': longitude, 'speed': speed})

mGps[['altitude','latitude','longitude','speed']] = mGps.apply(extract_gps_info, axis=1)
mGps['lifelog_date'] = mGps['timestamp'].astype(str).str[:10]
mGps = mGps.drop(columns=['m_gps'])
mGps.head(1)

,subject_id,timestamp,altitude,latitude,longitude,speed,lifelog_date
0,id01,2024-06-26 12:03:00,"[110.6, 110.8, 110.8, 110.7, 110.7, 110.8, 110.8, 110.8, 110.8, 110.8, 110.8]","[0.2077385, 0.2077759, 0.2077728, 0.20779, 0.2077914, 0.2077972, 0.2078002, 0.2077985, 0.207801, 0.207802, 0.2078011]","[0.170027, 0.1699851, 0.1699834, 0.1699686, 0.1699708, 0.1699657, 0.1699627, 0.1699631, 0.1699642, 0.1699639, 0.169963]","[0.0, 0.721, 0.0505, 0.6587, 0.0568, 0.1768, 0.0907, 0.0337, 0.0411, 0.0296, 0.0194]",2024-06-26


In [ ]:
# 거리 계산 함수
def haversine(coord1, coord2, unit='m'):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371000  # 지구 반지름(m)
    return c * r if unit == 'm' else c * r / 1000

def process_mGps(df):
    df = df.copy()

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['week'] = df['timestamp'].dt.isocalendar().week

    expanded_rows = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing GPS data"):
        speeds = ast.literal_eval(row['speed']) if isinstance(row['speed'], str) else row['speed']
        lats = ast.literal_eval(row['latitude']) if isinstance(row['latitude'], str) else row['latitude']
        lons = ast.literal_eval(row['longitude']) if isinstance(row['longitude'], str) else row['longitude']
        alts = ast.literal_eval(row['altitude']) if isinstance(row['altitude'], str) else row['altitude']
        n = len(speeds)
        if n > 0:
            expanded_rows.append(pd.DataFrame({
                'subject_id': [row['subject_id']] * n,
                'lifelog_date': [row['lifelog_date']] * n,
                'timestamp': pd.date_range(start=row['timestamp'], periods=n, freq='1S'),
                'speed': speeds,
                'latitude': lats,
                'longitude': lons,
                'altitude': alts
            }))

    expanded_df = pd.concat(expanded_rows, ignore_index=True)

    # 벡터화
    speeds = expanded_df['speed'].values

    walk_mask = (0.5 <= speeds) & (speeds < 2)
    jog_mask = (2 <= speeds) & (speeds < 5)
    vehicle_mask = (speeds >= 5)
    le5_mask = (speeds <= 5)

    expanded_df['walk'] = walk_mask.astype(int)
    expanded_df['jog'] = jog_mask.astype(int)
    expanded_df['vehicle'] = vehicle_mask.astype(int)
    expanded_df['le5_speed'] = expanded_df['speed'].where(le5_mask)

    # 아침/저녁 구간 조건
    expanded_df['hour'] = expanded_df['timestamp'].dt.hour
    morning_condition = (expanded_df['hour'] >= 6) & (expanded_df['hour'] < 9) & (expanded_df['speed'] >= 1)
    evening_condition = (expanded_df['hour'] >= 21) & (expanded_df['hour'] <= 23) & (expanded_df['speed'] <= 1)

    # 이동 특성 계산
    movement_features = []
    for (subject_id, lifelog_date), group in expanded_df.groupby(['subject_id', 'lifelog_date']):
        all_speeds = group['speed'].values
        all_alts = group['altitude'].values
        all_lats = group['latitude'].values
        all_lons = group['longitude'].values

        active_mins = group.shape[0] / 60  # 1초 단위 → 분
        movement_ratio = (all_speeds > 1.0).mean() if len(all_speeds) > 0 else 0
        alt_change = all_alts[-1] - all_alts[0] if len(all_alts) > 0 else 0
        lat_change = all_lats[-1] - all_lats[0] if len(all_lats) > 0 else 0
        lon_change = all_lons[-1] - all_lons[0] if len(all_lons) > 0 else 0

        total_dist = 0.0
        if len(all_lats) > 1:
            for i in range(len(all_lats)-1):
                coord1 = (all_lats[i], all_lons[i])
                coord2 = (all_lats[i+1], all_lons[i+1])
                total_dist += haversine(coord1, coord2, unit='m')

        movement_features.append({
            'subject_id': subject_id,
            'lifelog_date': lifelog_date,
            'active_minutes': active_mins,
            'movement_ratio': movement_ratio,
            'alt_change': alt_change,
            'lat_change': lat_change,
            'lon_change': lon_change,
            'total_distance_m': total_dist
        })

    movement_df = pd.DataFrame(movement_features)

    # Groupby + Aggregation
    agg_funcs = {
        'walk_minutes': ('walk', lambda x: x.sum() / 60),
        'jog_minutes': ('jog', lambda x: x.sum() / 60),
        'vehicle_minutes': ('vehicle', lambda x: x.sum() / 60),
        'speed_le5_max': ('le5_speed', 'max'),
        'speed_le5_mean': ('le5_speed', 'mean'),
        'speed_le5_std': ('le5_speed', 'std')
    }

    grouped = expanded_df.groupby(['subject_id', 'lifelog_date']).agg(**agg_funcs).reset_index()
    grouped['exercise_flag'] = np.where(grouped['jog_minutes'] >= 5,1,0)

    # 아침 wakeup time
    morning_first_movement = (
        expanded_df[morning_condition]
        .groupby(['subject_id', 'lifelog_date'])['timestamp']
        .min()
        .reset_index()
        .rename(columns={'timestamp': 'morning_wakeup_time'})
    )


    # 최종 merge
    final = pd.merge(grouped, movement_df, on=['subject_id', 'lifelog_date'], how='left')
    final = pd.merge(final, morning_first_movement, on=['subject_id', 'lifelog_date'], how='left')

    # 아침 wakeup_time 처리
    valid_wakeup = final['morning_wakeup_time'].dropna()
    if not valid_wakeup.empty:
        total_seconds = valid_wakeup.dt.hour * 3600 + valid_wakeup.dt.minute * 60 + valid_wakeup.dt.second
        mean_seconds = total_seconds.mean()
        mean_hour = int(mean_seconds // 3600)
        mean_minute = int((mean_seconds % 3600) // 60)
        mean_second = int(mean_seconds % 60)
        mean_wakeup_time = time(mean_hour, mean_minute, mean_second)
    else:
        mean_wakeup_time = time(7, 0, 0)

    final['morning_wakeup_time'] = final['morning_wakeup_time'].fillna(
        pd.Timestamp.combine(pd.to_datetime('today').date(), mean_wakeup_time)
    )
    final['morning_wakeup_time'] = final['morning_wakeup_time'].dt.hour * 100 + final['morning_wakeup_time'].dt.minute

    mean_wakeup_hhmm = mean_wakeup_time.hour * 100 + mean_wakeup_time.minute

    # wake_up_early_minutes
    def compute_minutes_diff(actual_hhmm, mean_hhmm):
        actual_hour = actual_hhmm // 100
        actual_minute = actual_hhmm % 100
        mean_hour = mean_hhmm // 100
        mean_minute = mean_hhmm % 100
        actual_sec = actual_hour * 3600 + actual_minute * 60
        mean_sec = mean_hour * 3600 + mean_minute * 60
        return (mean_sec - actual_sec) / 60

    final['wake_up_early_minutes'] = final['morning_wakeup_time'].apply(lambda x: compute_minutes_diff(x, mean_wakeup_hhmm))

    return final

In [ ]:
%%time

mGps2 = process_mGps(mGps)

# check
print(f'\n # mGps2 shape: {mGps2.shape}')
mGps2.head(1)

Processing GPS data: 100%|██████████| 800611/800611 [10:46<00:00, 1238.91it/s]



 # mGps2 shape: (660, 17)
CPU times: user 12min 4s, sys: 22.2 s, total: 12min 26s
Wall time: 12min 11s


,subject_id,lifelog_date,walk_minutes,jog_minutes,vehicle_minutes,speed_le5_max,speed_le5_mean,speed_le5_std,exercise_flag,active_minutes,movement_ratio,alt_change,lat_change,lon_change,total_distance_m,morning_wakeup_time,wake_up_early_minutes
0,id01,2024-06-26,11.1667,1.3000,3.8667,4.9907,0.2503,0.5089,0,100.2833,0.1034,-6.7000,0.0229,-0.0757,29113.5760,655,0.0000


### 🔥 mLight 주변 밝기
- Ambient light measured by the smartphone.
 - 어두운 밤	0.1 ~ 1 lux	캄캄한 방, 달빛 없는 밤
 - 가로등 켜진 거리	10 ~ 20 lux	흐릿한 외부 조명
 - 실내 조명	100 ~ 500 lux	사무실, 일반 거실
 - 밝은 실외	10,000 ~ 25,000 lux	맑은 날 햇빛
 - 직사광선 아래	30,000 ~ 100,000 lux	여름 한낮, 매우 강한 햇빛

- 밝기에 따라서 언제 불을 끄고 잠든 시간 추정
- 직사광선 잠에 좋은 영향을 주는지? (논문)
- 결측치 처리 x

In [ ]:
mLight['lifelog_date'] = mLight['timestamp'].astype(str).str[:10]
mLight.head(1)

,subject_id,timestamp,m_light,lifelog_date
0,id01,2024-06-26 12:03:00,534.0000,2024-06-26


In [ ]:
def process_mLight(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour
    df['is_night'] = df['hour'].apply(lambda h: h >= 22 or h < 6)

    # 하루 요약 통계
    daily_light = df.groupby(['subject_id', 'lifelog_date']).agg(
        light_mean=('m_light', 'mean'),
        light_std=('m_light', 'std'),
        light_max=('m_light', 'max'),
        light_min=('m_light', 'min'),
        light_night_mean=('m_light', lambda x: x[df.loc[x.index, 'is_night']].mean()),
        light_day_mean=('m_light', lambda x: x[~df.loc[x.index, 'is_night']].mean()),
        light_night_ratio=('is_night', 'mean')
    ).reset_index()

    results = []

    for subject_id, group in tqdm(df.groupby('subject_id'), desc="Processing light-based sleep detection"):
        group = group.sort_values('timestamp').reset_index(drop=True)

        recorded_dates = set()
        sleeping = False
        zero_count = 0
        first_zero_time = None

        for i in range(len(group)):
            light = group.loc[i, 'm_light']
            hour = group.loc[i, 'hour']

            if light == 0:
                zero_count += 1
                if zero_count == 1:
                    first_zero_time = group.loc[i, 'timestamp']
                if zero_count >= 6 and not sleeping:
                    sleep_hour = first_zero_time.hour
                    if (sleep_hour >= 21 or sleep_hour <= 2):
                        sleeping = True
            else:
                if sleeping:
                    candidate_wakeup = group.loc[i, 'timestamp']
                    wake_hour = candidate_wakeup.hour

                    if 5 <= wake_hour <= 9 and first_zero_time is not None:
                        wake_time = candidate_wakeup
                        sleep_time = first_zero_time
                        duration_min = (wake_time - sleep_time).total_seconds() / 60

                        if 0 < duration_min <= 840:
                            sleep_duration = duration_min
                        else:
                            sleep_duration = np.nan

                        lifelog_date = wake_time.date() + pd.Timedelta(days=-1)

                        if lifelog_date not in recorded_dates:
                            results.append({
                                'subject_id': subject_id,
                                'lifelog_date': lifelog_date,
                                'sleep_duration_min_mLight': sleep_duration,
                                'sleep_time_min_mLight': sleep_time.hour * 60 + sleep_time.minute,
                                'wake_time_min_mLight': wake_time.hour * 60 + wake_time.minute,
                                'hour_slept_mLight': sleep_time.hour + sleep_time.minute / 60,
                                'hour_woke_up_mLight': wake_time.hour + wake_time.minute / 60
                            })
                            recorded_dates.add(lifelog_date)

                        sleeping = False
                        zero_count = 0
                        first_zero_time = None

            if light > 0:
                zero_count = 0
                first_zero_time = None

    sleep_df = pd.DataFrame(results)

    # 정렬 + 보간
    sleep_df = sleep_df.sort_values(['subject_id', 'lifelog_date'])
    sleep_df['sleep_duration_interp_mLight'] = sleep_df.groupby('subject_id')['sleep_duration_min_mLight'].transform(lambda x: x.interpolate())

    # 시간 단위 파생 컬럼
    sleep_df['sleep_duration_hour_mLight'] = sleep_df['sleep_duration_min_mLight'] / 60
    sleep_df['sleep_duration_interp_hour_mLight'] = sleep_df['sleep_duration_interp_mLight'] / 60

    # 병합
    final = pd.merge(daily_light, sleep_df, on=['subject_id', 'lifelog_date'], how='left')

    return final

In [ ]:
mLight2 = process_mLight(mLight)

# check
print(f'\n # mLight2 shape: {mLight2.shape}')
mLight2.head(1)

Processing light-based sleep detection: 100%|██████████| 10/10 [00:01<00:00,  6.21it/s]


 # mLight2 shape: (700, 17)


,subject_id,lifelog_date,light_mean,light_std,light_max,light_min,light_night_mean,light_day_mean,light_night_ratio,sleep_duration_min_mLight,sleep_time_min_mLight,wake_time_min_mLight,hour_slept_mLight,hour_woke_up_mLight,sleep_duration_interp_mLight,sleep_duration_hour_mLight,sleep_duration_interp_hour_mLight
0,id01,2024-06-26,364.5068,395.6594,1886.0000,0.0000,184.9231,403.4167,0.1781,340.0000,1409.0000,309.0000,23.4833,5.1500,340.0000,5.6667,5.6667


### 🔥 mScreenStatus 화면 사용여부

- Indicates whether the smartphone screen is in use.
 - 기상시간, 취침시간, 수면시간
 - 휴대폰 이용횟수, 이용시간
 - 00 - 05 사이에 휴대폰 이용한 건수
 - 결측치 처리 x

In [ ]:
mScreenStatus['lifelog_date'] = mScreenStatus['timestamp'].astype(str).str[:10]
mScreenStatus.head(1)

,subject_id,timestamp,m_screen_use,lifelog_date
0,id01,2024-06-26 12:03:00,0,2024-06-26


In [ ]:
def preprocess_mScreenStatus(df):
    from datetime import datetime, time as dtime, timedelta

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])

    # base key 확보
    base_keys = df[['subject_id', 'lifelog_date']].drop_duplicates()
    base_keys['lifelog_date'] = base_keys['lifelog_date'].dt.date

    # 밤 9시부터 다음날 오전 11시 필터링
    df['hour'] = df['timestamp'].dt.hour
    df = df[(df['hour'] >= 21) | (df['hour'] < 11)].copy()
    df.loc[df['hour'] < 11, 'lifelog_date'] -= pd.Timedelta(days=1)

    df.sort_values(['subject_id', 'timestamp'], inplace=True)

    results = []

    for (subject_id, lifelog_date), group in df.groupby(['subject_id', 'lifelog_date']):
        group = group.sort_values('timestamp').reset_index(drop=True)

        # 1. 중간 각성(앞뒤 0, 본인 1) 제거
        prev = group['m_screen_use'].shift(1)
        next_ = group['m_screen_use'].shift(-1)
        mask = (group['m_screen_use'] == 1) & (prev == 0) & (next_ == 0)
        group.loc[mask, 'm_screen_use'] = 0

        # 2. 블록 단위로 짧은 각성 블록 제거
        group['is_sleep'] = group['m_screen_use'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()
        block_info = group.groupby('block').agg(
            is_sleep=('is_sleep', 'first'),
            size=('is_sleep', 'size')
        )

        for i in range(1, len(block_info) - 1):
            if (
                block_info.iloc[i]['is_sleep'] == False and
                block_info.iloc[i]['size'] <= 2 and
                block_info.iloc[i - 1]['is_sleep'] and
                block_info.iloc[i + 1]['is_sleep']
            ):
                group.loc[group['block'] == block_info.index[i], 'm_screen_use'] = 0

        # 다시 블록 재계산 후 수면 추정
        group['is_sleep'] = group['m_screen_use'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()
        sleep_blocks = group[group['is_sleep']].groupby('block').agg(
            sleep_start=('timestamp', 'first'),
            sleep_end=('timestamp', 'last'),
            duration_min=('timestamp', lambda x: (x.max() - x.min()).total_seconds() / 60)
        )

        sleep_time = wake_time = duration_min = None
        if not sleep_blocks.empty:
            longest_sleep = sleep_blocks.loc[sleep_blocks['duration_min'].idxmax()]
            sleep_time = longest_sleep['sleep_start'].time()
            wake_time = longest_sleep['sleep_end'].time()
            duration_min = (
                datetime.combine(datetime.today(), wake_time) - datetime.combine(datetime.today(), sleep_time)
            ).total_seconds() / 60
            if duration_min < 0:
                duration_min += 1440

            if not (4 <= wake_time.hour < 11):
                wake_time = None
            if not (sleep_time.hour >= 21 or sleep_time.hour < 3):
                sleep_time = None
            if duration_min < 100:
                sleep_time = None
                wake_time = None
                duration_min = None

        results.append({
            'subject_id': subject_id,
            'lifelog_date': lifelog_date.date(),
            'sleep_time': sleep_time,
            'wake_time': wake_time,
            'sleep_duration_min': round(duration_min, 1) if duration_min is not None else None
        })


    sleep_df = pd.DataFrame(results)
    result_df = base_keys.merge(sleep_df, on=['subject_id', 'lifelog_date'], how='left')

    # 시간 → 실수형 숫자 (예: 23:30 → 23.5)
    def time_to_float(t):
        if pd.isna(t):
            return None
        return round(t.hour + t.minute / 60 + t.second / 3600, 4)

    result_df['sleep_time'] = result_df['sleep_time'].apply(time_to_float)
    result_df['wake_time'] = result_df['wake_time'].apply(time_to_float)

    return result_df

In [ ]:
def preprocess_mScreenStatus(df):
    from datetime import datetime, timedelta

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])

    base_keys = df[['subject_id', 'lifelog_date']].drop_duplicates()
    base_keys['lifelog_date'] = base_keys['lifelog_date'].dt.date

    # 밤 9시 ~ 다음날 오전 11시 필터링
    df['hour'] = df['timestamp'].dt.hour
    df = df[(df['hour'] >= 21) | (df['hour'] < 11)].copy()
    df.loc[df['hour'] < 11, 'lifelog_date'] -= pd.Timedelta(days=1)
    df.sort_values(['subject_id', 'timestamp'], inplace=True)

    results = []

    for (subject_id, lifelog_date), group in df.groupby(['subject_id', 'lifelog_date']):
        group = group.sort_values('timestamp').reset_index(drop=True)

        # 중간 각성 제거
        prev = group['m_screen_use'].shift(1)
        next_ = group['m_screen_use'].shift(-1)
        mask = (group['m_screen_use'] == 1) & (prev == 0) & (next_ == 0)
        group.loc[mask, 'm_screen_use'] = 0

        # 짧은 각성 블록 제거
        group['is_sleep'] = group['m_screen_use'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()
        block_info = group.groupby('block').agg(
            is_sleep=('is_sleep', 'first'),
            size=('is_sleep', 'size')
        )

        for i in range(1, len(block_info) - 1):
            if (
                block_info.iloc[i]['is_sleep'] == False and
                block_info.iloc[i]['size'] <= 2 and
                block_info.iloc[i - 1]['is_sleep'] and
                block_info.iloc[i + 1]['is_sleep']
            ):
                group.loc[group['block'] == block_info.index[i], 'm_screen_use'] = 0

        # 블록 재계산
        group['is_sleep'] = group['m_screen_use'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()

        sleep_blocks = group[group['is_sleep']].groupby('block').agg(
            sleep_start=('timestamp', 'first'),
            sleep_end=('timestamp', 'last'),
            duration_min=('timestamp', lambda x: (x.max() - x.min()).total_seconds() / 60)
        )

        sleep_time = wake_time = duration_min = None
        if not sleep_blocks.empty:
            longest_sleep = sleep_blocks.loc[sleep_blocks['duration_min'].idxmax()]
            sleep_time = longest_sleep['sleep_start'].time()
            wake_time = longest_sleep['sleep_end'].time()
            duration_min = longest_sleep['duration_min']  # ✅ 정확하게 자정 넘는 경우도 반영됨

            # 유효 시간 범위 조건
            if not (4 <= wake_time.hour < 11):
                wake_time = None
            if not (sleep_time.hour >= 21 or sleep_time.hour < 3):
                sleep_time = None
            if duration_min < 100:
                sleep_time = None
                wake_time = None
                duration_min = None

        results.append({
            'subject_id': subject_id,
            'lifelog_date': lifelog_date.date(),
            'sleep_time': sleep_time,
            'wake_time': wake_time,
            'sleep_duration_min': round(duration_min, 1) if duration_min is not None else None
        })

    sleep_df = pd.DataFrame(results)
    result_df = base_keys.merge(sleep_df, on=['subject_id', 'lifelog_date'], how='left')

    # 시간 → 실수형 숫자 변환
    def time_to_float(t):
        if pd.isna(t):
            return None
        return round(t.hour + t.minute / 60 + t.second / 3600, 4)

    result_df['sleep_time'] = result_df['sleep_time'].apply(time_to_float)
    result_df['wake_time'] = result_df['wake_time'].apply(time_to_float)

    # 자정 넘어가는 경우 고려한 sleep_duration_min 재계산
    def compute_duration(row):
        sleep = row['sleep_time']
        wake = row['wake_time']
        if pd.isna(sleep) or pd.isna(wake):
            return None
        duration = (wake - sleep + 24) % 24
        return round(duration * 60, 1)  # 시간 단위 → 분 단위

    result_df['sleep_duration_min'] = result_df.apply(compute_duration, axis=1)

    return result_df

In [ ]:
def calculate_circular_mean_sleep_time(sleep_times):
    sleep_times = pd.Series(sleep_times).dropna()
    if len(sleep_times) == 0:
        return np.nan  # 혹은 return 0.0 등 기본값 설정 가능

    def hour_to_radian(hour):
        return (hour % 24) / 24 * 2 * np.pi

    radians = np.array([hour_to_radian(t) for t in sleep_times])
    mean_radian = np.arctan2(np.mean(np.sin(radians)), np.mean(np.cos(radians)))
    mean_hour = (mean_radian / (2 * np.pi)) * 24 % 24

    return mean_hour

In [ ]:
def circular_mean_sleep_time(times):

    # 결측치 제거
    valid_times = [t for t in times if pd.notna(t)]

    # 유효 데이터 개수 확인
    if len(valid_times) == 0:
        return None  # 결측치만 있는 경우

    # 시간 → 라디안 변환
    radians = [(t % 24) / 24 * 2 * np.pi for t in valid_times]

    # 사인/코사인 평균 계산
    sin_sum = np.mean(np.sin(radians))
    cos_sum = np.mean(np.cos(radians))

    # 평균 각도 계산
    if sin_sum == 0 and cos_sum == 0:
        return np.nan  # 불가능한 경우

    mean_radian = np.arctan2(sin_sum, cos_sum)

    # 평균 시간으로 변환
    mean_hour = (mean_radian / (2 * np.pi)) * 24
    if mean_hour < 0:
        mean_hour += 24

    return f'{int(mean_hour):02d}:{int((mean_hour % 1) * 60):02d}'

In [ ]:
def add_ratios(df):
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['weekday'] = df['lifelog_date'].dt.weekday  # 0=월 ~ 6=일
    df['week_type'] = df['weekday'].apply(lambda x: 'weekend' if x >= 5 else 'weekday')
    df['month'] = df['lifelog_date'].dt.month

    # 일반 평균 계산
    avg_duration = (
        df.groupby(['subject_id', 'month', 'week_type'])['sleep_duration_min']
        .mean()
        .reset_index(name='avg_sleep_duration')
    )

    # sleep_time, wake_time은 원형 평균 적용
    sleep_time_avg = (
        df.groupby(['subject_id', 'month', 'week_type'])['sleep_time']
        .apply(calculate_circular_mean_sleep_time)
        .reset_index(name='avg_sleep_time')
    )

    wake_time_avg = (
        df.groupby(['subject_id', 'month', 'week_type'])['wake_time']
        .apply(calculate_circular_mean_sleep_time)
        .reset_index(name='avg_wake_time')
    )

    # 평균값 합치기
    avg_df = sleep_time_avg.merge(wake_time_avg, on=['subject_id', 'month', 'week_type'])
    avg_df = avg_df.merge(avg_duration, on=['subject_id', 'month', 'week_type'])

    # 원본에 매칭
    df = df.merge(avg_df, on=['subject_id', 'month', 'week_type'], how='left')

    # 비율 변수 계산 (음수면 더 일찍 취침/기상/수면시간 짧음)
    df['sleep_time_diff'] = df['avg_sleep_time'] - df['sleep_time']
    df['wake_time_diff'] = df['avg_wake_time'] - df['wake_time']
    df['sleep_duration_diff'] = df['avg_sleep_duration'] - df['sleep_duration_min']
    df['sleep_time_ratio'] = df['sleep_time'] / df['avg_sleep_time']
    df['wake_time_ratio'] = df['wake_time'] / df['avg_wake_time']
    df['sleep_duration_ratio'] = df['sleep_duration_min'] / df['avg_sleep_duration']

    # lag feature
    df = df.sort_values(['subject_id', 'lifelog_date'])
    df['sleep_time_lag1'] = df.groupby('subject_id')['sleep_time'].shift(1)
    df['wake_time_lag1'] = df.groupby('subject_id')['wake_time'].shift(1)
    df['sleep_duration_lag1'] = df.groupby('subject_id')['sleep_duration_min'].shift(1)
    df['week_type_lag1'] = df.groupby('subject_id')['week_type'].shift(1)

    # 변화량
    df['sleep_time_diff_lag1'] = df.groupby('subject_id')['sleep_time'].diff()
    df['wake_time_diff_lag1'] = df.groupby('subject_id')['wake_time'].diff()
    df['sleep_duration_diff_lag1'] = df.groupby('subject_id')['sleep_duration_min'].diff()

    # 이동 평균 (3일)
    df['rolling_sleep_time_3d'] = (
        df.groupby('subject_id')['sleep_time']
        .rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)
    )
    df['rolling_wake_time_3d'] = (
        df.groupby('subject_id')['wake_time']
        .rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)
    )
    df['rolling_sleep_duration_3d'] = (
        df.groupby('subject_id')['sleep_duration_min']
        .rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)
    )

    # 존재하는 컬럼만 채우기
    existing_columns = df.columns.tolist()
    columns_to_fill_filtered = [col for col in [
        'sleep_time', 'wake_time', 'sleep_duration_min',
        'sleep_time_ratio', 'wake_time_ratio', 'sleep_duration_ratio',
        'sleep_time_diff', 'wake_time_diff', 'sleep_duration_diff',
        'sleep_time_lag1', 'wake_time_lag1', 'sleep_duration_lag1',
        'sleep_time_diff_lag1', 'wake_time_diff_lag1', 'sleep_duration_diff_lag1',
        'rolling_sleep_time_3d', 'rolling_wake_time_3d', 'rolling_sleep_duration_3d'
    ] if col in existing_columns]

    df[columns_to_fill_filtered] = df.groupby('subject_id')[columns_to_fill_filtered].ffill()

    result = df[[
        'subject_id', 'lifelog_date', 'week_type',
        'sleep_time', 'wake_time', 'sleep_duration_min',
        'avg_sleep_time', 'avg_wake_time', 'avg_sleep_duration',
        'sleep_time_ratio', 'wake_time_ratio', 'sleep_duration_ratio',
        'sleep_time_diff', 'wake_time_diff', 'sleep_duration_diff',
        'sleep_time_lag1','wake_time_lag1', 'sleep_duration_lag1','week_type_lag1',
        'sleep_time_diff_lag1','wake_time_diff_lag1','sleep_duration_diff_lag1',
        'rolling_sleep_time_3d','rolling_wake_time_3d','rolling_sleep_duration_3d'
    ]]

    return result

In [ ]:
mScreenStatus2 = preprocess_mScreenStatus(mScreenStatus)
mScreenStatus2 = add_ratios(mScreenStatus2)

# check
print(f'\n # mScreenStatus2 shape: {mScreenStatus2.shape}')
mScreenStatus2.head(1)


 # mScreenStatus2 shape: (700, 25)


,subject_id,lifelog_date,week_type,sleep_time,wake_time,sleep_duration_min,avg_sleep_time,avg_wake_time,avg_sleep_duration,sleep_time_ratio,wake_time_ratio,sleep_duration_ratio,sleep_time_diff,wake_time_diff,sleep_duration_diff,sleep_time_lag1,wake_time_lag1,sleep_duration_lag1,week_type_lag1,sleep_time_diff_lag1,wake_time_diff_lag1,sleep_duration_diff_lag1,rolling_sleep_time_3d,rolling_wake_time_3d,rolling_sleep_duration_3d
0,id01,2024-06-26,weekday,23.4500,5.2500,348.0000,23.1944,5.4887,377.6667,1.0110,0.9565,0.9214,-0.2556,0.2387,29.6667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.4500,5.2500,348.0000


In [ ]:
mScreenStatus2평균수면시간 = mScreenStatus2.groupby(['subject_id','week_type']).apply(lambda x: pd.Series({
     '평균 취침시간':circular_mean_sleep_time(x['sleep_time'])
    ,'평균 기상시간':circular_mean_sleep_time(x['wake_time'])
    ,'평균 수면시간':x['sleep_duration_min'].mean()
})).reset_index()

# 저장
fname = f'{path}mScreenStatus2평균수면시간.xlsx'
print(fname)
mScreenStatus2평균수면시간.to_excel(fname, index=False)

# check
mScreenStatus2평균수면시간

/content/drive/MyDrive/data/ch2025_data_items/mScreenStatus2평균수면시간.xlsx


,subject_id,week_type,평균 취침시간,평균 기상시간,평균 수면시간
0,id01,weekday,22:42,05:55,429.2292
1,id01,weekend,22:21,06:09,467.4500
2,id02,weekday,22:54,07:13,496.0000
3,id02,weekend,23:13,07:27,494.9583
4,id03,weekday,00:21,09:03,457.4359
5,id03,weekend,00:18,08:54,450.8667
6,id04,weekday,00:03,06:50,396.6721
7,id04,weekend,00:09,06:59,401.1739
8,id05,weekday,22:52,07:25,500.1064
9,id05,weekend,22:39,07:42,518.2778


### 🔥 mUsageStats 앱사용통계
- mUsageStats: Indicates which apps were used on the smartphone and for how long.

 - 몇시까지 핸드폰 보다가 잠잤는지
 - 통화, 전화 얼마나 했는지
 - YouTube 얼마나 봤는지
 - 메시지, 카카오톡 얼마나 했는지
 - NAVER 얼마나 했는지
 - 평소보다 얼마나 많은 앱을 이용했는지
 - 제외? -> 시스템 UI,One UI 홈

In [ ]:
def extract_mUsageStats_info(row):
    m_data = row['m_usage_stats']
    app_name = [item['app_name'] for item in m_data]
    total_time = [item['total_time'] for item in m_data]
    return pd.Series({'app_name': app_name, 'total_time': total_time})

mUsageStats[['app_name', 'total_time']] = mUsageStats.apply(extract_mUsageStats_info, axis=1)
mUsageStats['lifelog_date'] = mUsageStats['timestamp'].astype(str).str[:10]
mUsageStats.head(1)

,subject_id,timestamp,m_usage_stats,app_name,total_time,lifelog_date
0,id01,2024-06-26 13:00:00,"[{'app_name': ' 캐시워크', 'total_time': 69}, {'app_name': 'NAVER', 'total_time': 549}, {'app_name': ' ✝️성경일독Q', 'total_time': 7337}]","[ 캐시워크, NAVER, ✝️성경일독Q]","[69, 549, 7337]",2024-06-26


In [ ]:
def process_mUsageStats(df):
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['요일'] = df['lifelog_date'].dt.day_name()

    # 리스트 평탄화
    exploded_df = df.explode(['app_name', 'total_time'])
    exploded_df['total_time'] = exploded_df['total_time'].astype(float)
    exploded_df['total_time'] = exploded_df['total_time'] * 0.001 / 60  # 밀리초 → 초 → 분 변환

    # app_name 특수문자 제거
    exploded_df['app_name'] = exploded_df['app_name'].astype(str).apply(
        lambda x: re.sub(r'[^가-힣a-zA-Z0-9]', '', x)
    )

    # 시스템 앱 제거
    filtered_df = exploded_df[~exploded_df['app_name'].isin(['시스템UI'])]  # '시스템UI'만 제거 (OneUI홈은 포함)

    # 주요 파생변수 생성
    def calculate_daily_metrics(group):
        last_use = group['timestamp'].max()

        app_times = {
            '통화_time': group[group['app_name'] == '통화']['total_time'].sum(),
            '전화_time': group[group['app_name'] == '전화']['total_time'].sum(),
            'YouTube_time': group[group['app_name'] == 'YouTube']['total_time'].sum(),
            '메신저_time': group[group['app_name'].isin(['메시지', '카카오톡'])]['total_time'].sum(),
            'NAVER_time': group[group['app_name'] == 'NAVER']['total_time'].sum(),
            '캐시워크_time': group[group['app_name'] == '캐시워크']['total_time'].sum(),
            '성경일독Q_time': group[group['app_name'] == '성경일독Q']['total_time'].sum(),
            'OneUI홈_time': group[group['app_name'] == 'OneUI홈']['total_time'].sum(),
        }

        return pd.Series({
            **app_times,
            'unique_app_count': group['app_name'].nunique(),
            'total_screen_time': group['total_time'].sum()
        })

    # daily metrics 생성
    daily_stats = filtered_df.groupby(['subject_id','lifelog_date']).apply(calculate_daily_metrics).reset_index()

    # subject_id별 평균 총화면시간 구하기
    avg_screen_time = daily_stats.groupby('subject_id')['total_screen_time'].mean().to_dict()

    # 평균대비 화면사용량(%) 생성
    def compute_screen_usage(row):
        avg_time = avg_screen_time.get(row['subject_id'], np.nan)
        if pd.isna(avg_time) or avg_time == 0:
            return np.nan
        return round((row['total_screen_time'] / avg_time - 1) * 100, 1)

    daily_stats['screen_time_vs_avg_pct'] = daily_stats.apply(compute_screen_usage, axis=1)

    return daily_stats

In [ ]:
def process_mUsageStats(df):
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['요일'] = df['lifelog_date'].dt.day_name()
    df['hour'] = df['timestamp'].dt.hour

    # 시간대 분류
    def map_time_period(row):
        if 20 <= row['hour'] <= 23:
            return 'beforebed'
        else:
            return 'activehour'

    df['time_period'] = df.apply(map_time_period, axis=1)

    # 리스트 평탄화
    exploded_df = df.explode(['app_name', 'total_time'])
    exploded_df['total_time'] = exploded_df['total_time'].astype(float)
    exploded_df['total_time'] = exploded_df['total_time'] * 0.001 / 60  # 밀리초 → 초 → 분 변환

    # app_name 특수문자 제거
    exploded_df['app_name'] = exploded_df['app_name'].astype(str).apply(
        lambda x: re.sub(r'[^가-힣a-zA-Z0-9]', '', x)
    )

    # 시스템 앱 제거
    filtered_df = exploded_df[~exploded_df['app_name'].isin(['시스템UI'])]

    # 주요 파생변수 생성
    def calculate_daily_metrics(group):
        app_times = {
            '통화_time': group[group['app_name'] == '통화']['total_time'].sum(),
            '전화_time': group[group['app_name'] == '전화']['total_time'].sum(),
            'YouTube_time': group[group['app_name'] == 'YouTube']['total_time'].sum(),
            '메신저_time': group[group['app_name'].isin(['메시지', '카카오톡'])]['total_time'].sum(),
            'NAVER_time': group[group['app_name'] == 'NAVER']['total_time'].sum(),
            '캐시워크_time': group[group['app_name'] == '캐시워크']['total_time'].sum(),
            '성경일독Q_time': group[group['app_name'] == '성경일독Q']['total_time'].sum(),
            'OneUI홈_time': group[group['app_name'] == 'OneUI홈']['total_time'].sum(),
        }

        return pd.Series({
            **app_times,
            'unique_app_count': group['app_name'].nunique(),
            'total_screen_time': group['total_time'].sum()
        })

    # 일자/시간대별 요약
    daily_stats = filtered_df.groupby(['subject_id', 'lifelog_date', 'time_period']).apply(calculate_daily_metrics).reset_index()

    # subject_id별 평균 총화면시간
    avg_screen_time = daily_stats.groupby('subject_id')['total_screen_time'].mean().to_dict()

    # 평균 대비 비율
    def compute_screen_usage(row):
        avg_time = avg_screen_time.get(row['subject_id'], np.nan)
        if pd.isna(avg_time) or avg_time == 0:
            return np.nan
        return round((row['total_screen_time'] / avg_time - 1) * 100, 1)

    daily_stats['screen_time_vs_avg_pct'] = daily_stats.apply(compute_screen_usage, axis=1)

    # 피벗
    daily_stats = daily_stats.pivot(index=['subject_id', 'lifelog_date'], columns='time_period')
    daily_stats.columns = [f"{tp}_{metric}" for metric, tp in daily_stats.columns]
    daily_stats = daily_stats.reset_index()

    return daily_stats

In [ ]:
mUsageStats2 = process_mUsageStats(mUsageStats)

# check
print(f'\n # mUsageStats2 shape: {mUsageStats2.shape}')
mUsageStats2.head(1)


 # mUsageStats2 shape: (689, 24)


,subject_id,lifelog_date,activehour_통화_time,beforebed_통화_time,activehour_전화_time,beforebed_전화_time,activehour_YouTube_time,beforebed_YouTube_time,activehour_메신저_time,beforebed_메신저_time,activehour_NAVER_time,beforebed_NAVER_time,activehour_캐시워크_time,beforebed_캐시워크_time,activehour_성경일독Q_time,beforebed_성경일독Q_time,activehour_OneUI홈_time,beforebed_OneUI홈_time,activehour_unique_app_count,beforebed_unique_app_count,activehour_total_screen_time,beforebed_total_screen_time,activehour_screen_time_vs_avg_pct,beforebed_screen_time_vs_avg_pct
0,id01,2024-06-26,9.0010,0.2079,11.3007,0.7731,0.1061,0.0000,43.6359,14.5713,8.4852,0.1351,18.6694,5.4722,88.3836,27.6892,61.1160,27.9861,25.0000,20.0000,266.7672,156.8681,-29.0000,-58.3000


### ✔️ mWifi 주변wifi 정보
- Wifi devices around individual subject.
 - -30 ~ -50 dBm	매우 강한 신호 (최적)
 - -51 ~ -60 dBm	강한 신호 (문제 없음)
 - -61 ~ -70 dBm	괜찮은 신호 (약간 느릴 수 있음)
 - -71 ~ -80 dBm	약한 신호 (끊김 주의)
 - -81 dBm 이하	매우 약한 신호 (거의 끊김)

In [ ]:
def extract_wifi_info(row):
    wifi_data = row['m_wifi']
    bssids = [item['bssid'] for item in wifi_data]
    rssis = [item['rssi'] for item in wifi_data]
    return pd.Series({'bssid': bssids, 'rssi': rssis})

mWifi[['bssid', 'rssi']] = mWifi.apply(extract_wifi_info, axis=1)
mWifi['lifelog_date'] = mWifi['timestamp'].astype(str).str[:10]
mWifi.head(1)

,subject_id,timestamp,m_wifi,bssid,rssi,lifelog_date
0,id01,2024-06-26 12:03:00,"[{'bssid': 'a0:0f:37:9a:5d:8b', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8c', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8d', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8e', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8f', 'rssi': -78}, {'bssid': 'a0:0f:37:96:56:ef', 'rssi': -58}, {'bssid': '88:36:6c:86:75:84', 'rssi': -72}, {'bssid': 'a0:0f:37:96:56:ee', 'rssi': -58}, {'bssid': 'a0:0f:37:96:56:ed', 'rssi': -58}, {'bssid': '86:25:19:b5:b2:a5', 'rssi': -61}, {'bssid': 'a0:0f:37:96:56:ec', 'rssi': -58}, {'bssid': '1e:39:29:8e:fb:e9', 'rssi': -71}, {'bssid': '52:c2:e8:c7:9b:e4', 'rssi': -82}, {'bssid': 'a0:0f:37:96:56:eb', 'rssi': -58}, {'bssid': '12:e3:c7:09:20:34', 'rssi': -88}, {'bssid': '58:86:94:4a:08:b8', 'rssi': -82}, {'bssid': '90:9f:33:28:d0:2e', 'rssi': -78}, {'bssid': '00:26:66:bc:4e:18', 'rssi': -85}, {'bssid': 'f6:0a:f4:43:4b:ba', 'rssi': -45}, {'bssid': '10:e3:c7:09:20:35', 'rssi': -63}, {'bssid': '10:e3:c7:09:20:34', 'rssi': -89}, {'bssid': '1c:39:29:48:04:92', 'rssi': -82}, {'bssid': '12:e3:c7:07:9d:df', 'rssi': -83}, {'bssid': '86:25:19:c3:44:07', 'rssi': -84}, {'bssid': 'a0:0f:37:9a:37:2f', 'rssi': -76}, {'bssid': 'a0:0f:37:9a:37:2e', 'rssi': -76}, {'bssid': 'a0:0f:37:9a:37:2d', 'rssi': -76}, {'bssid': '0a:09:b4:74:05:ec', 'rssi': -72}, {'bssid': 'a0:0f:37:9a:37:2c', 'rssi': -76}, {'bssid': 'a0:0f:37:9a:37:2b', 'rssi': -76}, {'bssid': '0a:09:b4:74:05:eb', 'rssi': -59}, {'bssid': 'c0:25:2f:d8:c1:a6', 'rssi': -82}, {'bssid': '16:7f:67:bb:fa:f8', 'rssi': -79}, {'bssid': '3c:f3:92:ff:00:01', 'rssi': -82}, {'bssid': '06:09:b4:74:05:ec', 'rssi': -72}, {'bssid': '06:09:b4:74:05:eb', 'rssi': -59}, {'bssid': '12:e3:c7:0a:74:d1', 'rssi': -78}, {'bssid': '88:36:6c:a9:6f:8e', 'rssi': -63}, {'bssid': '02:e3:c7:09:20:34', 'rssi': -88}, {'bssid': '00:09:b4:74:05:eb', 'rssi': -60}, {'bssid': '00:09:b4:74:05:ec', 'rssi': -72}, {'bssid': '00:1d:93:93:cf:fe', 'rssi': -19}, {'bssid': '8e:e2:ac:a5:9d:15', 'rssi': -72}]","[a0:0f:37:9a:5d:8b, a0:0f:37:9a:5d:8c, a0:0f:37:9a:5d:8d, a0:0f:37:9a:5d:8e, a0:0f:37:9a:5d:8f, a0:0f:37:96:56:ef, 88:36:6c:86:75:84, a0:0f:37:96:56:ee, a0:0f:37:96:56:ed, 86:25:19:b5:b2:a5, a0:0f:37:96:56:ec, 1e:39:29:8e:fb:e9, 52:c2:e8:c7:9b:e4, a0:0f:37:96:56:eb, 12:e3:c7:09:20:34, 58:86:94:4a:08:b8, 90:9f:33:28:d0:2e, 00:26:66:bc:4e:18, f6:0a:f4:43:4b:ba, 10:e3:c7:09:20:35, 10:e3:c7:09:20:34, 1c:39:29:48:04:92, 12:e3:c7:07:9d:df, 86:25:19:c3:44:07, a0:0f:37:9a:37:2f, a0:0f:37:9a:37:2e, a0:0f:37:9a:37:2d, 0a:09:b4:74:05:ec, a0:0f:37:9a:37:2c, a0:0f:37:9a:37:2b, 0a:09:b4:74:05:eb, c0:25:2f:d8:c1:a6, 16:7f:67:bb:fa:f8, 3c:f3:92:ff:00:01, 06:09:b4:74:05:ec, 06:09:b4:74:05:eb, 12:e3:c7:0a:74:d1, 88:36:6c:a9:6f:8e, 02:e3:c7:09:20:34, 00:09:b4:74:05:eb, 00:09:b4:74:05:ec, 00:1d:93:93:cf:fe, 8e:e2:ac:a5:9d:15]","[-78, -78, -78, -78, -78, -58, -72, -58, -58, -61, -58, -71, -82, -58, -88, -82, -78, -85, -45, -63, -89, -82, -83, -84, -76, -76, -76, -72, -76, -76, -59, -82, -79, -82, -72, -59, -78, -63, -88, -60, -72, -19, -72]",2024-06-26


In [ ]:
def process_mWifi(df,threshold):

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    def filter_strong_rssi(df,threshold):
        filtered_df = df.copy()
        def filter_row(row):
            bssids = row['bssid']
            rssis = row['rssi']
            # RSSI > threshold 조건 만족하는 항목만 추출
            filtered = [(b, r) for b, r in zip(bssids, rssis) if r > threshold]
            if filtered:
                new_bssids, new_rssis = zip(*filtered)
                return pd.Series({'bssid': list(new_bssids), 'rssi': list(new_rssis)})
            else:
                return pd.Series({'bssid': [], 'rssi': []})
        filtered_df[['bssid', 'rssi']] = filtered_df.apply(filter_row, axis=1)
        return filtered_df

    # === wifi 약신호 제거 ===
    df = filter_strong_rssi(df, threshold=threshold) ####

    features = []
    grouped = df.groupby(['subject_id', 'lifelog_date'])

    for (subject_id, date), group in grouped:
        scan_count = len(group)
        bssid_flat = sum(group['bssid'], [])  # flatten
        rssi_flat = sum(group['rssi'], [])    # flatten

        unique_bssid_count = len(set(bssid_flat))
        avg_rssi = sum(rssi_flat) / len(rssi_flat) if rssi_flat else None
        max_rssi = max(rssi_flat) if rssi_flat else None
        min_rssi = min(rssi_flat) if rssi_flat else None
        strong_rssi_ratio = sum(1 for r in rssi_flat if r > -60) / len(rssi_flat) if rssi_flat else 0
        empty_scan_count = sum(1 for b in group['bssid'] if len(b) == 0)

        # 가장 많이 탐지된 BSSID
        bssid_counter = Counter(bssid_flat)
        top_bssid, top_bssid_count = bssid_counter.most_common(1)[0] if bssid_counter else (None, 0)

        first_time = group['timestamp'].min()
        last_time = group['timestamp'].max()
        hour_span = (last_time - first_time).total_seconds() / 60  # 분 단위

        features.append({
            'subject_id': subject_id,
            'lifelog_date': date,
            'scan_count': scan_count,
            'unique_bssid_count': unique_bssid_count,
            'avg_rssi': avg_rssi,
            'max_rssi': max_rssi,
            # 'min_rssi': min_rssi,
            # 'strong_signal_ratio': strong_rssi_ratio,
            'empty_scan_count': empty_scan_count,
            'top_bssid': top_bssid,
            'top_bssid_count': top_bssid_count,
            'hour_span_minutes': hour_span
        })

    return pd.DataFrame(features)

In [ ]:
mWifi2 = process_mWifi(mWifi,threshold=-60)

# check
print(f'\n # mWifi2 shape: {mWifi2.shape}')
mWifi2.head(1)


 # mWifi2 shape: (685, 10)


,subject_id,lifelog_date,scan_count,unique_bssid_count,avg_rssi,max_rssi,empty_scan_count,top_bssid,top_bssid_count,hour_span_minutes
0,id01,2024-06-26,69,48,-49.6109,-19.0000,11,86:25:19:9f:9b:be,19,716.0000


### ✔️ wHr 심박동수
- Heart rate readings recorded by the smartwatch.


In [ ]:
wHr['lifelog_date'] = wHr['timestamp'].astype(str).str[:10]
wHr.head(1)

,subject_id,timestamp,heart_rate,lifelog_date
0,id01,2024-06-26 12:23:00,"[134, 134, 135, 133, 134, 135, 134, 135, 134, 133, 133, 133, 132, 132, 131, 131, 131, 132, 132, 134, 134, 134, 132, 130, 128, 126, 126, 126, 127, 129, 130, 129, 130, 130, 127, 127, 126, 125, 123]",2024-06-26


In [ ]:
def get_time_block(hour):
    if 0 <= hour < 6:
        return 'early_morning'
    elif 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'afternoon'
    else:
        return 'evening'

def process_wHr_by_timeblock(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['block'] = df['timestamp'].dt.hour.map(get_time_block)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'lifelog_date']):
        block_stats = {'subject_id': subj, 'lifelog_date': date}

        for block, block_group in group.groupby('block'):
            hr_all = []
            for row in block_group['heart_rate']:
                parsed = ast.literal_eval(row) if isinstance(row, str) else row
                hr_all.extend([int(h) for h in parsed if h is not None])

            if not hr_all:
                continue

            above_100 = [hr for hr in hr_all if hr > 100]
            block_stats[f'hr_{block}_mean'] = np.mean(hr_all)
            block_stats[f'hr_{block}_std'] = np.std(hr_all)
            block_stats[f'hr_{block}_max'] = np.max(hr_all)
            block_stats[f'hr_{block}_min'] = np.min(hr_all)
            block_stats[f'hr_{block}_above_100_ratio'] = len(above_100) / len(hr_all)

        results.append(block_stats)

    return pd.DataFrame(results)

In [ ]:
wHr2 = process_wHr_by_timeblock(wHr)

# check
print(f'\n # wHr2 shape: {wHr2.shape}')
wHr2.head(1)


 # wHr2 shape: (636, 22)


,subject_id,lifelog_date,hr_afternoon_mean,hr_afternoon_std,hr_afternoon_max,hr_afternoon_min,hr_afternoon_above_100_ratio,hr_evening_mean,hr_evening_std,hr_evening_max,hr_evening_min,hr_evening_above_100_ratio,hr_early_morning_mean,hr_early_morning_std,hr_early_morning_max,hr_early_morning_min,hr_early_morning_above_100_ratio,hr_morning_mean,hr_morning_std,hr_morning_max,hr_morning_min,hr_morning_above_100_ratio
0,id01,2024-06-26,80.5333,12.6366,142.0000,59.0000,0.0773,82.4768,10.2932,124.0000,59.0000,0.0555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### ✔️ wLight 앰비언트 라이트
- Ambient light measured by the smartwatch.  
  - 어두운 밤 0.1 ~ 1 lux 캄캄한 방, 달빛 없는 밤
  - 가로등 켜진 거리 10 ~ 20 lux 흐릿한 외부 조명
  - 실내 조명 100 ~ 500 lux 사무실, 일반 거실
  - 밝은 실외 10,000 ~ 25,000 lux 맑은 날 햇빛
  - 직사광선 아래 30,000 ~ 100,000 lux 여름 한낮, 매우 강한 햇빛

In [ ]:
wLight['lifelog_date'] = wLight['timestamp'].astype(str).str[:10]
wLight.head(1)

,subject_id,timestamp,w_light,lifelog_date
0,id01,2024-06-26 12:17:00,633.0000,2024-06-26


In [ ]:
def get_time_block(hour):
    if 0 <= hour < 6:
        return 'early_morning'
    elif 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'afternoon'
    else:
        return 'evening'

def process_wLight_by_timeblock(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['block'] = df['timestamp'].dt.hour.map(get_time_block)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'lifelog_date']):
        block_stats = {'subject_id': subj, 'lifelog_date': date}

        for block, block_group in group.groupby('block'):
            lux = block_group['w_light'].dropna().values
            if len(lux) == 0:
                continue

            block_stats[f'wlight_{block}_mean'] = np.mean(lux)
            block_stats[f'wlight_{block}_std'] = np.std(lux)
            block_stats[f'wlight_{block}_max'] = np.max(lux)
            block_stats[f'wlight_{block}_min'] = np.min(lux)

        results.append(block_stats)

    return pd.DataFrame(results)

In [ ]:
wLight2 = process_wLight_by_timeblock(wLight)

# check
print(f'\n # wLight2 shape: {wLight2.shape}')
wLight2.head(1)


 # wLight2 shape: (664, 18)


,subject_id,lifelog_date,wlight_afternoon_mean,wlight_afternoon_std,wlight_afternoon_max,wlight_afternoon_min,wlight_evening_mean,wlight_evening_std,wlight_evening_max,wlight_evening_min,wlight_early_morning_mean,wlight_early_morning_std,wlight_early_morning_max,wlight_early_morning_min,wlight_morning_mean,wlight_morning_std,wlight_morning_max,wlight_morning_min
0,id01,2024-06-26,394.5251,1458.7346,20874.0000,0.0000,89.0202,101.6844,264.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### ✔️ wPedo 걸음수
- Step data recorded by the smartwatch.

In [ ]:
wPedo['lifelog_date'] = wPedo['timestamp'].astype(str).str[:10]
wPedo.head(1)

,subject_id,timestamp,step,step_frequency,running_step,walking_step,distance,speed,burned_calories,lifelog_date
0,id01,2024-06-26 12:09:00,10,0.1667,0,0,8.3300,0.1388,0.0000,2024-06-26


In [ ]:
def process_wPedo(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date

    summary = df.groupby(['subject_id', 'lifelog_date']).agg({
        'step': 'sum',
        'step_frequency': 'mean',
        'distance': 'sum',
        'speed': ['mean', 'max'],
        'burned_calories': 'sum'
    }).reset_index()

    # 컬럼 이름 정리
    summary.columns = ['subject_id', 'lifelog_date',
                       'step_sum', 'step_frequency_mean',
                       'distance_sum', 'speed_mean', 'speed_max',
                       'burned_calories_sum']

    return summary

In [ ]:
wPedo2 = process_wPedo(wPedo)

# check
print(f'\n # wPedo2 shape: {wPedo2.shape}')
wPedo2.head(1)


 # wPedo2 shape: (653, 8)


,subject_id,lifelog_date,step_sum,step_frequency_mean,distance_sum,speed_mean,speed_max,burned_calories_sum
0,id01,2024-06-26,3578,0.0927,2782.1901,0.0721,1.5882,189.3191


### 🔥 운동 추정 파생변수

- mActivity 추정행동
- mGps, 핸드폰 위치
- wHr 심박동수
- wPedo 걸음수

In [ ]:
def average_list_columns(df, list_columns, pk_cols=['subject_id', 'lifelog_date']):

    for col in list_columns:

        def safe_mean(x):
            if isinstance(x, list):
                return np.mean(x) if len(x) > 0 else np.nan
            elif isinstance(x, (int, float, np.integer, np.floating, type(None))):
                return x
            elif isinstance(x, (np.ndarray, pd.Series)):
                return np.mean(x)
            elif pd.api.types.is_scalar(x) and pd.isna(x):
                return np.nan
            else:
                return np.nan

        df[col] = df[col].apply(safe_mean)

    return df

def compute_estimated_exercise(mActivity, mGps, wHr, wPedo, minutes):

    # 리스트 평균 처리
    mGps = mGps.copy()
    wHr = wHr.copy()
    mGps = average_list_columns(mGps, ['speed'])
    Hr = average_list_columns(wHr, ['heart_rate'])

    for df in [mActivity, mGps, wHr, wPedo]:
        df['timestamp'] = pd.to_datetime(df['timestamp'])

    # 5분 지속 조건 판단 함수
    def sustained_condition(df, cond_col,minutes):
        df = df[df[cond_col]].sort_values('timestamp')
        times = df['timestamp']
        start = prev = None
        for t in times:
            if start is None:
                start = prev = t
            elif t <= prev + timedelta(minutes=1):
                prev = t
            else:
                if prev - start >= timedelta(minutes=minutes):
                    return True
                start = prev = t
        return (prev - start) >= timedelta(minutes=minutes) if start else False

    # mActivity: m_activity == 7 지속
    mActivity['m_cond'] = mActivity['m_activity'] == 7
    act_flag = mActivity.groupby(['subject_id', 'lifelog_date']) \
                        .apply(lambda df: sustained_condition(df, 'm_cond',40)) \
                        .reset_index(name='act_exe_flag')

    # mGps: speed ∈ [2.5, 5.5] 지속
    mGps['gps_cond'] = mGps['speed'].between(2.5, 5.5)
    gps_flag = mGps.groupby(['subject_id', 'lifelog_date']) \
                   .apply(lambda df: sustained_condition(df, 'gps_cond',minutes)) \
                   .reset_index(name='gps_exe_flag')

    # wHr: hr ≥ 133 상태가 5분 이상 유지
    wHr['whr_cond'] = wHr['heart_rate'] >= 133
    hr_flag = wHr.groupby(['subject_id', 'lifelog_date']) \
                   .apply(lambda df: sustained_condition(df, 'whr_cond',minutes)) \
                   .reset_index(name='hr_exe_flag')

    # wPedo: step ≥ 10000 또는 running_step ≥ 1이 5분 이상
    pedo_daily = wPedo.groupby(['subject_id', 'lifelog_date'])['step'].sum().reset_index(name='total_steps')
    pedo_daily['step_flag'] = pedo_daily['total_steps'] >= 10000

    wPedo['r_cond'] = wPedo['running_step'] >= 1
    run_flag = wPedo.groupby(['subject_id', 'lifelog_date']) \
                    .apply(lambda df: sustained_condition(df, 'r_cond', minutes)) \
                    .reset_index(name='run_flag')

    pedo_flag = pedo_daily.merge(run_flag, on=['subject_id', 'lifelog_date'], how='outer')
    pedo_flag['step_flag'] = pedo_flag['step_flag'].fillna(False)
    pedo_flag['run_flag'] = pedo_flag['run_flag'].fillna(False)
    pedo_flag['pedo_exe_flag'] = pedo_flag[['step_flag', 'run_flag']].any(axis=1)

    # 병합 및 최종 판단
    result = act_flag.merge(gps_flag, on=['subject_id', 'lifelog_date'], how='outer') \
                     .merge(hr_flag, on=['subject_id', 'lifelog_date'], how='outer') \
                     .merge(pedo_flag[['subject_id', 'lifelog_date', 'pedo_exe_flag']], on=['subject_id', 'lifelog_date'], how='outer')

    # NaN 처리 및 1/0 변환
    for col in ['act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag']:
        result[col] = result[col].fillna(False)

    result['pred_exe_flag'] = result[['act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag']].any(axis=1)

    # 👉 1/0 변환
    for col in ['act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag', 'pred_exe_flag']:
        result[col] = result[col].astype(int)

    display(result[['pred_exe_flag', 'act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag']].sum())

    return result[['subject_id', 'lifelog_date', 'pred_exe_flag', 'act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag']]

In [ ]:
# 추정운동여부
exeFlag = compute_estimated_exercise(mActivity, mGps, wHr, wPedo,10)

# check
print(f'\n # exeFlag shape: {exeFlag.shape}')
exeFlag.head(1)

,0
pred_exe_flag,66
act_exe_flag,31
gps_exe_flag,5
hr_exe_flag,18
pedo_exe_flag,19



 # exeFlag shape: (700, 7)


,subject_id,lifelog_date,pred_exe_flag,act_exe_flag,gps_exe_flag,hr_exe_flag,pedo_exe_flag
0,id01,2024-06-26,0,0,0,0,0


### 🔥 Sleeptime 일어난 건수

- Sleeptime에 (mLight 주변 밝기), (wLight 앰비언트 라이트) 변화 건수

In [ ]:
def compute_night_awake_features(df, prefix):

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # 00시~06시 필터
    df['hour'] = df['timestamp'].dt.hour
    df_night = df[(df['hour'] >= 0) & (df['hour'] < 6)].copy()

    # 깨어있는 분 계산
    df_night['awake_minute'] = (df_night[prefix] > 0).astype(int)

    # 깨어난 횟수 계산 (0 → 양수 전환)
    def count_awake_blocks(x):
        return ((x > 0) & (x.shift(fill_value=0) == 0)).sum()

    # 그룹별 집계
    result = df_night.groupby(['subject_id', 'lifelog_date']).agg(
        awake_minutes=('awake_minute', 'sum'),
        awake_blocks=(prefix, count_awake_blocks)
    ).reset_index()

    # 컬럼명 변경
    result = result.rename(columns={
        'awake_minutes': f'{prefix}_awake_minutes',
        'awake_blocks': f'{prefix}_awake_blocks'
    })

    # train에 결과 합치기 위해서 -1 day 하기
    result['lifelog_date'] = pd.to_datetime(result['lifelog_date'])
    result['lifelog_date'] = result['lifelog_date'] + pd.Timedelta(days=-1)

    result['lifelog_date'] = result['lifelog_date'].astype(str)

    return result

In [ ]:
a1 = compute_night_awake_features(mLight,'m_light')
a2 = compute_night_awake_features(wLight,'w_light')
sleepWakeCnt = train[['subject_id','lifelog_date']].copy()

sleepWakeCnt = sleepWakeCnt.merge(a1, on=['subject_id','lifelog_date'], how='left')
sleepWakeCnt = sleepWakeCnt.merge(a2, on=['subject_id','lifelog_date'], how='left')

sleepWakeCnt['awake_minutes'] = sleepWakeCnt[['m_light_awake_minutes','w_light_awake_minutes']].max(axis=1)
sleepWakeCnt['awake_blocks'] = sleepWakeCnt[['m_light_awake_blocks','w_light_awake_blocks']].max(axis=1)

# check
sleepWakeCnt.head()

,subject_id,lifelog_date,m_light_awake_minutes,m_light_awake_blocks,w_light_awake_minutes,w_light_awake_blocks,awake_minutes,awake_blocks
0,id01,2024-06-26,4.0000,1.0000,17.0000,3.0000,17.0000,3.0000
1,id01,2024-06-27,4.0000,1.0000,14.0000,3.0000,14.0000,3.0000
2,id01,2024-06-28,4.0000,1.0000,0.0000,0.0000,4.0000,1.0000
3,id01,2024-06-29,1.0000,1.0000,0.0000,0.0000,1.0000,1.0000
4,id01,2024-06-30,2.0000,1.0000,0.0000,0.0000,2.0000,1.0000


### 📦 merge 데이터
- train, test 기간 서로 겹침

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/data/ch2025_metrics_train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/ch2025_submission_sample.csv')

# 일자변수 타입 변환
mACStatus2['lifelog_date'] = mACStatus2['lifelog_date'].astype(str)
mActivity2['lifelog_date'] = mActivity2['lifelog_date'].astype(str)
mAmbience2['lifelog_date'] = mAmbience2['lifelog_date'].astype(str)
mBle2['lifelog_date'] = mBle2['lifelog_date'].astype(str)
mGps2['lifelog_date'] = mGps2['lifelog_date'].astype(str)
mLight2['lifelog_date'] = mLight2['lifelog_date'].astype(str)
mScreenStatus2['lifelog_date'] = mScreenStatus2['lifelog_date'].astype(str)
mUsageStats2['lifelog_date'] = mUsageStats2['lifelog_date'].astype(str)
mWifi2['lifelog_date'] = mWifi2['lifelog_date'].astype(str)
wHr2['lifelog_date'] = wHr2['lifelog_date'].astype(str)
wLight2['lifelog_date'] = wLight2['lifelog_date'].astype(str)
wPedo2['lifelog_date'] = wPedo2['lifelog_date'].astype(str)

# ---- new ----

exeFlag['lifelog_date'] = exeFlag['lifelog_date'].astype(str)
sleepWakeCnt['lifelog_date'] = sleepWakeCnt['lifelog_date'].astype(str)

In [ ]:
df_list = [
    mACStatus2,       # 1
    mActivity2,       # 2
    mAmbience2,       # 3
    mBle2,            # 4
    mGps2,            # 5
    mLight2,          # 6
    mScreenStatus2,   # 7
    mUsageStats2,     # 8
    mWifi2,           # 9
    wHr2,             # 10
    wLight2,          # 11
    wPedo2,           # 12
    # ---- new ----
    sleepWakeCnt,
    exeFlag
]

data = reduce(lambda left, right: pd.merge(left, right, on=['subject_id', 'lifelog_date'], how='outer'), df_list)
data['lifelog_date'] = data['lifelog_date'].astype(str)

# 중복체크
print(data.shape)
print(data[['subject_id','lifelog_date']].drop_duplicates().shape)

# merge
train2 = train.merge(data, on=['subject_id','lifelog_date'], how='left')
test2 = test.merge(data, on=['subject_id','lifelog_date'], how='left')

# 저장
print('# train  shape:',train.shape)
print('# train2 shape:',test2.shape)
print('# test   shape:',test.shape)
print('# test2  shape:',test2.shape)

In [ ]:
# 저장
train2.to_parquet(f"/content/drive/MyDrive/data/train_0512.parquet")
test2.to_parquet(f"/content/drive/MyDrive/data/test_0512.parquet")

# 📌 모델 학습

In [ ]:
train2 = pd.read_parquet(f"/content/drive/MyDrive/data/train_0512.parquet")
test2 = pd.read_parquet(f"/content/drive/MyDrive/data/test_0512.parquet")

In [ ]:
train = train2.copy()
test = test2.copy()

# drop_features = ['afterwork_max_label','sleeptime_max_label','worktime_max_label']
drop_features = ['top_bssid'] # ,'week_type','week_type_lag1'
drop_features = [i for i in drop_features if i in train.columns.tolist()]
print('# drop_features:',drop_features)
train = train.drop(columns=drop_features)
test = test.drop(columns=drop_features)

# drop_features: ['top_bssid']


In [ ]:
# 요일 컬럼 추가 (예: 월요일, 화요일, ...)
train['lifelog_date'] = pd.to_datetime(train['lifelog_date'])
test['lifelog_date'] = pd.to_datetime(test['lifelog_date'])

# 요일
weekday_map = {
    0: '월요일', 1: '화요일', 2: '수요일', 3: '목요일',
    4: '금요일', 5: '토요일', 6: '일요일'
}
train['weekday'] = train['lifelog_date'].dt.dayofweek.map(weekday_map)
test['weekday'] = test['lifelog_date'].dt.dayofweek.map(weekday_map)

# 월
train['month'] = train['lifelog_date'].dt.month
test['month'] = test['lifelog_date'].dt.month

# weekend
train['weekend'] = np.where(train['weekday'].isin(['토요일','일요일']),1,0)
test['weekend'] = np.where(test['weekday'].isin(['토요일','일요일']),1,0)

# 공휴일
공휴일 = [
     '2024-08-15'
    ,'2024-09-16'
    ,'2024-09-17'
    ,'2024-09-18'
    ,'2024-10-03'
    ,'2024-10-09'
]
train['공휴일'] = np.where(train['lifelog_date'].isin(공휴일),1,0)
test['공휴일'] = np.where(test['lifelog_date'].isin(공휴일),1,0)

# 주말 + 공휴일 묶어주기
# train['weekend'] = np.where( ((train['weekend']==0) & (train['공휴일']==1)), 1, train['weekend'])
# test['weekend'] = np.where( ((test['weekend']==0) & (test['공휴일']==1)), 1, test['weekend'])

# 숫자형 컬럼만 선택해서 결측값 -1로 채우기
train[train.select_dtypes(include='number').columns] = train.select_dtypes(include='number').fillna(-1)
test[test.select_dtypes(include='number').columns] = test.select_dtypes(include='number').fillna(-1)

In [ ]:
def find_best_threshold(y_true, y_proba):
    thresholds = np.linspace(0.05, 0.95, 91)
    best_f1 = 0
    best_thresh = 0.5
    for t in thresholds:
        preds = (y_proba >= t).astype(int)
        score = f1_score(y_true, preds)
        if score > best_f1:
            best_f1 = score
            best_thresh = t
    return best_thresh, best_f1

def add_noise(series, noise_level, seed=3):
    rng = np.random.default_rng(seed)
    return series * (1 + noise_level * rng.standard_normal(len(series)))

def calculate_averages(data,name):
    variables = ['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']
    variable_averages = {}
    total_sum = 0
    total_count = 0

    for var in variables:
        values = []
        for entry in data.values():
            if var in entry:  # 키가 존재하는 경우에만 추가
                values.append(entry[var])
        avg = sum(values) / len(values) if values else None  # 누락된 변수 처리
        variable_averages[var] = round(avg, 6) if avg is not None else 'Missing'
        total_sum += sum(values)
        total_count += len(values)

    overall_avg = round(total_sum / total_count, 6) if total_count > 0 else None
    print(f'# 전체 평균 {name}: {overall_avg} {variable_averages}')

In [69]:
def run_basemodel(train, test, valid_ids, common_params, random_state, best_thresh_dict, submit=False):

    train_df = train.copy()
    test_df = test.copy()
    submission_final = test_df[['subject_id', 'sleep_date', 'lifelog_date']].copy()
    submission_final['lifelog_date'] = pd.to_datetime(submission_final['lifelog_date']).dt.date

    # 타겟
    targets_binary = ['Q1', 'Q2', 'Q3', 'S2', 'S3']
    targets_binary_name = ['기상직후수면질','취침전신체적피로','취침전스트레스','수면효율','수면잠들기시간']
    target_multiclass = 'S1'
    all_targets = targets_binary + [target_multiclass]

    # ========
    # 타겟인코딩
    # ========

    # 노이즈 수준 설정
    noise_level = 0.015

    for tgt in all_targets:

      encoder_feats = ['subject_id','month','weekend']

      #### 타겟인코딩1

      subject_mean = train_df.groupby(encoder_feats)[tgt].mean().rename(f'{tgt}_te')
      train_df = train_df.merge(subject_mean, on=encoder_feats, how='left')
      test_df = test_df.merge(subject_mean, on=encoder_feats, how='left')
      global_mean = train_df[tgt].mean()
      test_df[f'{tgt}_te'] = test_df[f'{tgt}_te'].fillna(global_mean)

      # 노이즈 추가
      train_df[f'{tgt}_te'] = add_noise(train_df[f'{tgt}_te'], noise_level)
      test_df[f'{tgt}_te'] = add_noise(test_df[f'{tgt}_te'], noise_level)

      #### 타겟인코딩2

      # 새로운 범주형 열 생성
      train_df['TMP'] = train_df[encoder_feats].applymap(str).apply(lambda x: ''.join(x) ,axis=1)
      test_df['TMP'] = test_df[encoder_feats].applymap(str).apply(lambda x: ''.join(x) ,axis=1)

      # 인코더
      encoder = TargetEncoder(cols=['TMP'], smoothing=300) # 40
      encoder.fit(train_df[['TMP']], train_df[tgt])

      # 인코딩 결과를 새로운 열에 저장
      train_df[f'{tgt}_te2'] = encoder.transform(train_df[['TMP']])
      test_df[f'{tgt}_te2'] = encoder.transform(test_df[['TMP']])

      # 노이즈 추가
      train_df[f'{tgt}_te2'] = add_noise(train_df[f'{tgt}_te2'], noise_level)
      test_df[f'{tgt}_te2'] = add_noise(test_df[f'{tgt}_te2'], noise_level)

      # 불필요한 변수 제거
      train_df = train_df.drop(columns=['TMP'])
      test_df = test_df.drop(columns=['TMP'])

    # 인코딩
    PK = ['sleep_date', 'lifelog_date', 'subject_id']
    encoder = LabelEncoder()
    categorical_features = [i for i in train_df.select_dtypes(include=['object', 'category']).columns if i not in PK+['pk']]
    print(f'# categorical_features: {categorical_features}')
    for col in categorical_features:
        train_df[col] = encoder.fit_transform(train_df[col])
        test_df[col] = encoder.fit_transform(test_df[col])

    # X
    X = train_df.drop(columns=PK + all_targets)
    test_X = test_df.drop(columns=PK + all_targets)

    # ======
    # binary
    # ======

    val_f1 = {}
    val_logloss = {}
    best_thresh_dict = {}
    for col in targets_binary:

        # data split
        y = train_df[col]
        train_df['pk'] = train_df['subject_id']+train_df['sleep_date']
        X_valid = train_df.loc[train_df['pk'].isin(valid_ids),X.columns.tolist()].reset_index(drop=True).copy()
        X_train = train_df.loc[~train_df['pk'].isin(valid_ids),X.columns.tolist()].reset_index(drop=True).copy()
        y_valid = train_df.loc[train_df['pk'].isin(valid_ids),y.name].reset_index(drop=True).copy()
        y_train = train_df.loc[~train_df['pk'].isin(valid_ids),y.name].reset_index(drop=True).copy()

        # seed
        best_param = common_params[col].copy()
        best_param['random_state'] = random_state

        # model train
        model = LGBMClassifier(**best_param)
        model.fit(X_train, y_train)

        # predict & eval
        # pred_valid = model.predict(X_valid)
        # f1 = f1_score(y_valid, pred_valid, average='macro') ### 수정
        # val_f1.append(f1)

        # 예측
        proba_valid = model.predict_proba(X_valid)[:, 1]
        best_thresh, _ = find_best_threshold(y_valid, proba_valid) # 최적 threshold 탐색
        pred_valid = (proba_valid >= best_thresh).astype(int) # threshold 적용
        best_thresh_dict[col] = float(best_thresh)

        # 평가 지표
        loss = log_loss(y_valid, proba_valid)
        f1 = f1_score(y_valid, pred_valid, average='macro')
        val_f1[col] = f1
        val_logloss[col] = loss

        # check
        print(f'# target:{col}, logloss:{np.round(loss,4)}, f1:{np.round(f1,4)}, best_thresh:{np.round(best_thresh,4)}')

    # ======
    # multi
    # ======

    # data split
    y = train_df[target_multiclass]
    X_valid = train_df.loc[train_df['pk'].isin(valid_ids),X.columns.tolist()].reset_index(drop=True).copy()
    X_train = train_df.loc[~train_df['pk'].isin(valid_ids),X.columns.tolist()].reset_index(drop=True).copy()
    y_valid = train_df.loc[train_df['pk'].isin(valid_ids),y.name].reset_index(drop=True).copy()
    y_train = train_df.loc[~train_df['pk'].isin(valid_ids),y.name].reset_index(drop=True).copy()

    # seed
    best_param = common_params['S1'].copy()
    best_param['random_state'] = random_state

    # model train
    model = LGBMClassifier(**best_param, objective='multiclass', num_class=3)
    model.fit(X_train, y_train)

    # predict & eval
    pred_valid = model.predict(X_valid)
    f1 = f1_score(y_valid, pred_valid, average='macro')
    val_f1['S1'] = f1

    # ==============
    # binary + multi
    # ==============

    avg_f1 = np.mean(list(val_f1.values()))
    avg_logloss = np.mean(list(val_logloss.values()))
    detail_f1 = " ".join([f"{name}({tname}):{score:.4f}" for name, tname, score in zip(targets_binary + [target_multiclass], targets_binary_name + ['수면시간'], list(val_f1.values()))])
    detail_logloss = " ".join([f"{name}({tname}):{score:.4f}" for name, tname, score in zip(targets_binary + [target_multiclass], targets_binary_name + ['수면시간'], list(val_logloss.values()))])
    print(f"# 평균 F1: {avg_f1:.4f} / [상세] {detail_f1}")
    print(f"# 평균 logloss: {avg_logloss:.4f} / [상세] {detail_logloss}")

    # ======
    # submit
    # ======

    if submit==True:

      print('\n# (submit)전체 데이터로 모델 재학습..')

      # binary
      # binary_preds = {}
      for col in targets_binary:
          binary_params = common_params[col].copy()
          y = train_df[col]
          model = LGBMClassifier(**binary_params)
          model.fit(X, y)
          # binary_preds[col] = model.predict(test_X)
          # fi_df = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
          # top10 = fi_df.sort_values(by='importance', ascending=False).head(10)
          # feat_str = ", ".join([f"{row['feature']}({int(row['importance'])})" for _, row in top10.iterrows()])
          # print(f"[{col}] {feat_str}")

          # 예측
          proba_test = model.predict_proba(test_X)[:, 1]
          pred_test = (proba_test >= best_thresh_dict[col]).astype(int) # threshold 적용
          submission_final[col] = pred_test

          # vi[1]
          fi_df = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
          top10 = fi_df.sort_values(by='importance', ascending=False).head(10)
          feat_str = ", ".join([f"{row['feature']}({int(row['importance'])})" for _, row in top10.iterrows()])
          print(f"[{col}] {feat_str}")

      # multiclass
      y = train_df['S1']
      binary_params = common_params['S1'].copy()
      model = LGBMClassifier(**binary_params)
      model.fit(X, y)
      multiclass_pred = model.predict(test_X)
      submission_final['S1'] = multiclass_pred

      # vi[2]
      fi_df = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
      top10 = fi_df.sort_values(by='importance', ascending=False).head(10)
      feat_str = ", ".join([f"{row['feature']}({int(row['importance'])})" for _, row in top10.iterrows()])
      print(f"[S1] {feat_str}")

      # 예측 저장
      submission_final = submission_final[['subject_id', 'sleep_date', 'lifelog_date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]
      fname = f"/content/drive/MyDrive/data/submission_{avg_f1}_{avg_logloss}.csv"
      submission_final.to_csv(fname, index=False)
      print(f"# {fname} 저장 완료")

      # 모델별 예측결과 비율 비교
      a11 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].sum()
      a13 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].apply(len)
      a12 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].mean()
      a21 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].sum()
      a23 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].apply(len)
      a22 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].mean()
      result = pd.concat([a11, a13, a12, a21, a23, a22], axis=1)
      result.columns = ['학습sum','학습len','학습mean','테스트sum','테스트len','테스트mean']
      print('\n# 예측결과 비교표')
      display(result)

      # 클래스 비율 계산
      a1 = train['S1'].value_counts(normalize=True).sort_index().rename('train_ratio')
      a2 = submission_final['S1'].value_counts(normalize=True).sort_index().rename('test_ratio')
      merged_dist = pd.concat([a1, a2], axis=1).fillna(0)
      merged_dist = merged_dist.round(3)
      print("\n # S1 클래스별 비율 (Train vs Test)")
      display(merged_dist)

    return submission_final, val_f1, val_logloss, best_thresh_dict

In [70]:
%%time

# 공통 하이퍼파라미터
common_params = {
  'n_estimators': 5000,
  'learning_rate': 0.01,
  'lambda_l1': 5,
  'lambda_l2': 1,
  'n_jobs': -1,
  'verbosity': -1,
  # ----------------------- 랜덤속성
  'bagging_fraction': 0.9, # 1
  'bagging_freq': 1,     # 1
  'feature_fraction': 0.9, # 1
}

# 모델별 세부 하이퍼파라미터
best_param_dict = {}

# 공통 하이퍼파라미터 대체 (이상한 모델의 경우)
best_param_dict['Q3'] = common_params
best_param_dict['S1'] = common_params
best_param_dict['S2'] = common_params
best_param_dict['S3'] = common_params
best_param_dict['Q1'] = common_params
best_param_dict['Q2'] = common_params

# 실험
random_state = 41
f1_val_dict = {}
logloss_val_dict = {}
thresh_val_dict = {}
for i,valid_ids in enumerate([valid_ids1,valid_ids2,valid_ids3,valid_ids4]):
  print(f"\n")
  submission,f1_val_dict[i],logloss_val_dict[i],thresh_val_dict[i] = run_basemodel(train, test, valid_ids, best_param_dict, random_state, None, submit=False) # random_state=41

# check
print(f"\n")
calculate_averages(f1_val_dict,name='F1')
calculate_averages(logloss_val_dict,name='logloss')
calculate_averages(thresh_val_dict,name='임계값')

# 임계값 설정
best_thresh_dict = {**thresh_val_dict[0], **thresh_val_dict[1], **thresh_val_dict[2], **thresh_val_dict[3]}
print(f'\n# best_thresh_dict: {best_thresh_dict}')

# 제출
submission, _, _, _ = run_basemodel(train, test, valid_ids1, best_param_dict, random_state, best_thresh_dict, submit=True)



# categorical_features: ['week_type', 'week_type_lag1', 'weekday']
# target:Q1, logloss:0.6052, f1:0.7382, best_thresh:0.4
# target:Q2, logloss:0.5714, f1:0.7351, best_thresh:0.45
# target:Q3, logloss:0.5978, f1:0.5333, best_thresh:0.26
# target:S2, logloss:0.6471, f1:0.6438, best_thresh:0.55
# target:S3, logloss:0.5369, f1:0.734, best_thresh:0.5
# 평균 F1: 0.6412 / [상세] Q1(기상직후수면질):0.7382 Q2(취침전신체적피로):0.7351 Q3(취침전스트레스):0.5333 S2(수면효율):0.6438 S3(수면잠들기시간):0.7340 S1(수면시간):0.4629
# 평균 logloss: 0.5917 / [상세] Q1(기상직후수면질):0.6052 Q2(취침전신체적피로):0.5714 Q3(취침전스트레스):0.5978 S2(수면효율):0.6471 S3(수면잠들기시간):0.5369


# categorical_features: ['week_type', 'week_type_lag1', 'weekday']
# target:Q1, logloss:0.5325, f1:0.7704, best_thresh:0.54
# target:Q2, logloss:0.6043, f1:0.5545, best_thresh:0.15
# target:Q3, logloss:0.5728, f1:0.6769, best_thresh:0.37
# target:S2, logloss:0.5128, f1:0.6854, best_thresh:0.34
# target:S3, logloss:0.617, f1:0.7615, best_thresh:0.62
# 평균 F1: 0.6557 / [상세] Q1(기상직후수면질):0.7704 Q

,학습sum,학습len,학습mean,테스트sum,테스트len,테스트mean
Q1,223,450,0.4956,158,250,0.6320
Q2,253,450,0.5622,167,250,0.6680
Q3,270,450,0.6000,212,250,0.8480
S1,390,450,0.8667,206,250,0.8240
S2,293,450,0.6511,155,250,0.6200
S3,298,450,0.6622,169,250,0.6760



 # S1 클래스별 비율 (Train vs Test)


,train_ratio,test_ratio
S1,,
0,0.3180,0.3080
1,0.4980,0.5600
2,0.1840,0.1320


CPU times: user 9min 54s, sys: 2.2 s, total: 9min 56s
Wall time: 1min 25s


In [ ]:
"""
==================
[benchmark]
valid*: 0.6359
oof***: 0.6620
public: 0.62832
==================

# categorical_features: ['week_type', 'week_type_lag1', 'weekday']
# random_state: 41
# 평균 F1: 0.6475 / [상세] Q1(기상직후수면질):0.7030 Q2(취침전신체적피로):0.7405 Q3(취침전스트레스):0.6489 S2(수면효율):0.5955 S3(수면잠들기시간):0.7340 S1(수면시간):0.4629
# 전체 평균 F1: 0.6475

# categorical_features: ['week_type', 'week_type_lag1', 'weekday']
# random_state: 41
# 평균 F1: 0.6546 / [상세] Q1(기상직후수면질):0.7143 Q2(취침전신체적피로):0.6847 Q3(취침전스트레스):0.6652 S2(수면효율):0.6870 S3(수면잠들기시간):0.6907 S1(수면시간):0.4857
# 전체 평균 F1: 0.6546

# categorical_features: ['week_type', 'week_type_lag1', 'weekday']
# random_state: 41
# 평균 F1: 0.6486 / [상세] Q1(기상직후수면질):0.7522 Q2(취침전신체적피로):0.7172 Q3(취침전스트레스):0.6329 S2(수면효율):0.6500 S3(수면잠들기시간):0.6665 S1(수면시간):0.4729
# 전체 평균 F1: 0.6486

# categorical_features: ['week_type', 'week_type_lag1', 'weekday']
# random_state: 41
# 평균 F1: 0.6532 / [상세] Q1(기상직후수면질):0.7395 Q2(취침전신체적피로):0.6636 Q3(취침전스트레스):0.6684 S2(수면효율):0.6769 S3(수면잠들기시간):0.7166 S1(수면시간):0.4540
# 전체 평균 F1: 0.6532

# <최종 평균 F1>: 0.6509598675639194

# categorical_features: ['week_type', 'week_type_lag1', 'weekday']
# random_state: 41
# 평균 F1: 0.6475 / [상세] Q1(기상직후수면질):0.7030 Q2(취침전신체적피로):0.7405 Q3(취침전스트레스):0.6489 S2(수면효율):0.5955 S3(수면잠들기시간):0.7340 S1(수면시간):0.4629
# 전체 평균 F1: 0.6475

# (submit)전체 데이터로 모델 재학습..
[Q1] Q1_te2(794), light_night_mean(475), wake_time_ratio(466), Q1_te(436), wake_time_diff_lag1(427), sleep_duration_diff(334), ble_rssi_max_afterwork(322), beforebed_통화_time(302), activehour_total_screen_time(294), hr_evening_std(291)
[Q2] Q2_te2(990), rolling_wake_time_3d(560), Q2_te(470), wake_up_early_minutes(434), sleep_duration_min(420), avg_rssi(383), rolling_sleep_time_3d(379), speed_le5_max(339), wlight_evening_mean(314), hr_evening_std(292)
[Q3] Q3_te2(942), light_max(561), Q3_te(508), activehour_통화_time(473), sleep_duration_diff_lag1(426), sleep_time(347), total_distance_m(326), all_VEHICLE_minutes(323), activehour_OneUI홈_time(310), activehour_unique_app_count(306)
[S2] S2_te(928), S2_te2(751), beforebed_total_screen_time(408), wake_time_diff_lag1(391), light_max(387), activehour_screen_time_vs_avg_pct(361), sleep_duration_min(345), avg_rssi(336), activehour_unique_label_count(323), sleep_duration_ratio(303)
[S3] S3_te(923), S3_te2(491), light_night_mean(457), ble_class_unknwn_ratio_sleeptime(418), activehour_unique_label_count(409), ble_rssi_mean_afterwork(378), vehicle_minutes(370), beforebed_메신저_time(370), lon_change(331), hr_morning_mean(313)
[S1] S1_te2(1118), S1_te(930), sleep_duration_min(881), beforebed_screen_time_vs_avg_pct(869), wake_time_ratio(796), sleep_duration_ratio(791), hour_span_minutes(762), beforebed_total_screen_time(715), scan_count(655), speed_le5_max(629)
# /content/drive/MyDrive/data/submission_0.6474751826606326.csv 저장 완료

# 예측결과 비교표
학습sum	학습len	학습mean	테스트sum	테스트len	테스트mean
Q1	223	450	0.4956	133	250	0.5320
Q2	253	450	0.5622	158	250	0.6320
Q3	270	450	0.6000	170	250	0.6800
S1	390	450	0.8667	206	250	0.8240
S2	293	450	0.6511	164	250	0.6560
S3	298	450	0.6622	169	250	0.6760

 # S1 클래스별 비율 (Train vs Test)
train_ratio	test_ratio
S1
0	0.3180	0.3080
1	0.4980	0.5600
2	0.1840	0.1320
CPU times: user 11min 44s, sys: 2.33 s, total: 11min 47s
Wall time: 1min 37s
"""